# Preparation
## Imports

In [ ]:
%config IPCompleter.greedy=True
from IPython.display import display, HTML

In [ ]:
import os
import warnings
import logging
import pickle
import sys
import time
import collections
import re
from itertools import groupby
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import sklearn
import seaborn as sns

from sklearn.base import clone

# pre-processing

from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.utils import resample
from sklearn.preprocessing import KBinsDiscretizer
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, SVMSMOTE, BorderlineSMOTE, SMOTENC
from imblearn.over_sampling import ADASYN

# classifiers

from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC


# Feature selection

from sklearn.feature_selection import RFECV, SelectFromModel, SelectPercentile, SelectKBest, chi2, mutual_info_classif

# learning and cross-validation

from sklearn.model_selection import ParameterGrid, StratifiedShuffleSplit, StratifiedKFold, RepeatedStratifiedKFold, \
                                    cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.metrics import make_scorer, get_scorer, f1_score, precision_score, recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report   
from sklearn.metrics import roc_curve, auc
from sklearn import metrics


try:
    del sys.modules['feutils']
    del sys.modules['config']
except:
    pass
import config
from feutils import DataHandler
from feutils import JNCC2Wrapper
from feutils import FEUtils

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
logger = logging.getLogger()
warnings.filterwarnings("ignore")

feu = FEUtils()

print('pandas ' + str(pd.__version__))

## Sklearn customizations

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator

class PassthroughTransformer(BaseEstimator, TransformerMixin):
    """
    sklearn' default transformer does not give access to feature names, and Pipeline may change
    order of features. This implementation overloads default passthrough transformer so it returns features names.
    """
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        self.X = X
        return X
    
    def get_feature_names(self):
        return self.X.columns.tolist()

In [ ]:
from sklearn.utils import check_array
from sklearn.utils.extmath import _incremental_mean_and_var
from sklearn.utils.validation import (check_is_fitted, check_random_state,
                                FLOAT_DTYPES)
from sklearn.preprocessing.data import _handle_zeros_in_scale
from scipy import sparse

class CustomStandardScaler(BaseEstimator, TransformerMixin):
    """Customized to add 'get_feature_names'
    """  # noqa

    def __init__(self, copy=True, with_mean=True, with_std=True):
        self.with_mean = with_mean
        self.with_std = with_std
        self.copy = copy

    def _reset(self):
        """Reset internal data-dependent state of the scaler, if necessary.
        __init__ parameters are not touched.
        """

        # Checking one attribute is enough, becase they are all set together
        # in partial_fit
        if hasattr(self, 'scale_'):
            del self.scale_
            del self.n_samples_seen_
            del self.mean_
            del self.var_

    def fit(self, X, y=None):
        """Compute the mean and std to be used for later scaling.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape [n_samples, n_features]
            The data used to compute the mean and standard deviation
            used for later scaling along the features axis.
        y
            Ignored
        """

        # Reset internal state before fitting
        self._reset()
        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns
        return self.partial_fit(X, y)

    def partial_fit(self, X, y=None):
        """Online computation of mean and std on X for later scaling.
        All of X is processed as a single batch. This is intended for cases
        when `fit` is not feasible due to very large number of `n_samples`
        or because X is read from a continuous stream.
        The algorithm for incremental mean and std is given in Equation 1.5a,b
        in Chan, Tony F., Gene H. Golub, and Randall J. LeVeque. "Algorithms
        for computing the sample variance: Analysis and recommendations."
        The American Statistician 37.3 (1983): 242-247:
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape [n_samples, n_features]
            The data used to compute the mean and standard deviation
            used for later scaling along the features axis.
        y
            Ignored
        """
        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns        
        X = check_array(X, accept_sparse=('csr', 'csc'), copy=self.copy,
                        estimator=self, dtype=FLOAT_DTYPES,
                        force_all_finite='allow-nan')

        # Even in the case of `with_mean=False`, we update the mean anyway
        # This is needed for the incremental computation of the var
        # See incr_mean_variance_axis and _incremental_mean_variance_axis

        # if n_samples_seen_ is an integer (i.e. no missing values), we need to
        # transform it to a NumPy array of shape (n_features,) required by
        # incr_mean_variance_axis and _incremental_variance_axis
        if (hasattr(self, 'n_samples_seen_') and
                isinstance(self.n_samples_seen_, (int, np.integer))):
            self.n_samples_seen_ = np.repeat(
                self.n_samples_seen_, X.shape[1]).astype(np.int64, copy=False)

        if sparse.issparse(X):
            if self.with_mean:
                raise ValueError(
                    "Cannot center sparse matrices: pass `with_mean=False` "
                    "instead. See docstring for motivation and alternatives.")

            sparse_constructor = (sparse.csr_matrix
                                  if X.format == 'csr' else sparse.csc_matrix)
            counts_nan = sparse_constructor(
                        (np.isnan(X.data), X.indices, X.indptr),
                        shape=X.shape).sum(axis=0).A.ravel()

            if not hasattr(self, 'n_samples_seen_'):
                self.n_samples_seen_ = (
                        X.shape[0] - counts_nan).astype(np.int64, copy=False)

            if self.with_std:
                # First pass
                if not hasattr(self, 'scale_'):
                    self.mean_, self.var_ = mean_variance_axis(X, axis=0)
                # Next passes
                else:
                    self.mean_, self.var_, self.n_samples_seen_ = \
                        incr_mean_variance_axis(X, axis=0,
                                                last_mean=self.mean_,
                                                last_var=self.var_,
                                                last_n=self.n_samples_seen_)
            else:
                self.mean_ = None
                self.var_ = None
                if hasattr(self, 'scale_'):
                    self.n_samples_seen_ += X.shape[0] - counts_nan
        else:
            if not hasattr(self, 'n_samples_seen_'):
                self.n_samples_seen_ = np.zeros(X.shape[1], dtype=np.int64)

            # First pass
            if not hasattr(self, 'scale_'):
                self.mean_ = .0
                if self.with_std:
                    self.var_ = .0
                else:
                    self.var_ = None

            if not self.with_mean and not self.with_std:
                self.mean_ = None
                self.var_ = None
                self.n_samples_seen_ += X.shape[0] - np.isnan(X).sum(axis=0)
            else:
                self.mean_, self.var_, self.n_samples_seen_ = \
                    _incremental_mean_and_var(X, self.mean_, self.var_,
                                              self.n_samples_seen_)

        # for backward-compatibility, reduce n_samples_seen_ to an integer
        # if the number of samples is the same for each feature (i.e. no
        # missing values)
        if np.ptp(self.n_samples_seen_) == 0:
            self.n_samples_seen_ = self.n_samples_seen_[0]

        if self.with_std:
            self.scale_ = _handle_zeros_in_scale(np.sqrt(self.var_))
        else:
            self.scale_ = None

        return self

    def transform(self, X, copy=None):
        """Perform standardization by centering and scaling
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data used to scale along the features axis.
        copy : bool, optional (default: None)
            Copy the input X or not.
        """
        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns        
        check_is_fitted(self, 'scale_')

        copy = copy if copy is not None else self.copy
        X = check_array(X, accept_sparse='csr', copy=copy,
                        estimator=self, dtype=FLOAT_DTYPES,
                        force_all_finite='allow-nan')

        if sparse.issparse(X):
            if self.with_mean:
                raise ValueError(
                    "Cannot center sparse matrices: pass `with_mean=False` "
                    "instead. See docstring for motivation and alternatives.")
            if self.scale_ is not None:
                inplace_column_scale(X, 1 / self.scale_)
        else:
            if self.with_mean:
                X -= self.mean_
            if self.with_std:
                X /= self.scale_
        self.X = X
        return X

    def inverse_transform(self, X, copy=None):
        """Scale back the data to the original representation
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data used to scale along the features axis.
        copy : bool, optional (default: None)
            Copy the input X or not.
        Returns
        -------
        X_tr : array-like, shape [n_samples, n_features]
            Transformed array.
        """
        check_is_fitted(self, 'scale_')

        copy = copy if copy is not None else self.copy
        if sparse.issparse(X):
            if self.with_mean:
                raise ValueError(
                    "Cannot uncenter sparse matrices: pass `with_mean=False` "
                    "instead See docstring for motivation and alternatives.")
            if not sparse.isspmatrix_csr(X):
                X = X.tocsr()
                copy = False
            if copy:
                X = X.copy()
            if self.scale_ is not None:
                inplace_column_scale(X, self.scale_)
        else:
            X = np.asarray(X)
            if copy:
                X = X.copy()
            if self.with_std:
                X *= self.scale_
            if self.with_mean:
                X += self.mean_
        return X

    def _more_tags(self):
        return {'allow_nan': True}
    
    def get_feature_names(self):
        return self.feature_names
    
def parse_feature_names(feature_names):
    reg = r'.*__(.*)'
    return [re.findall(reg, feat)[0] for feat in feature_names]

## Data loading

In [ ]:
# IPUs for agent and new way of computing IPUs for doctor/phases
matrix_subject_nophase = os.path.join('..', 'features', 'new_ipus', 'matrix-2019-0607_00-25-49Features-nophase.xlsx')
matrix_agent_phases = os.path.join('..', 'features', 'new_ipus', 'matrix-2019-0624_11-38-41Features-agent-157015.xlsx')
matrix_subject_phases = os.path.join('..', 'features', 'new_ipus', 'matrix-2019-0624_12-13-32Features-157015.xlsx')
matrix_agent_nophase = os.path.join('..', 'features', 'new_ipus', 'matrix-2019-0624_11-41-11Features-agent-nophase.xlsx')

# new features angles speeds means/stds
#matrix_subject_nophase = os.path.join('..', 'features', 'angles', 'matrix-2019-0607_00-25-49Features-nophase.xlsx')
#matrix_agent_phases = os.path.join('..', 'features', 'angles', 'matrix-2019-0608_12-31-38Features-agent-157015.xlsx')
#matrix_subject_phases = os.path.join('..', 'features', 'angles', 'matrix-2019-0608_08-46-42Features-157015.xlsx')
#matrix_agent_nophase = os.path.join('..', 'features', 'angles', 'matrix-2019-0608_15-03-58Features-agent-nophase.xlsx')


# new feature 'answerDelay'
"""matrix_agent_phases = os.path.join('..', 'features', 'answersDelays', matrix-2019-0529_13-58-51Features-agent-157015.xlsx')
matrix_subject_phases = os.path.join('..', 'features', 'answersDelays', 'matrix-2019-0529_11-20-12Features-157015.xlsx')
trix_subject_nophase = os.path.join('..', 'features', 'answersDelays', 'matrix-2019-0602_22-16-02Features-nophase.xlsx')
matrix_agent_nophase = os.path.join('..', 'features', 'answersDelays', 'matrix-2019-0602_21-45-19Features-agent-nophase.xlsx')"""

# 'cleaned' dataset
"""matrix_subject_phases = os.path.join('..', 'features', 'cleaned', 'matrix-2019-0409_14-52-24Features-157015.xlsx')
matrix_agent_phases = os.path.join('..', 'features', 'cleaned', 'matrix-2019-0410_17-50-48Features-agent-157015.xlsx')
matrix_subject_nophase = os.path.join('..', 'features', 'cleaned', 'matrix-2019-0411_14-05-24Features-nophase.xlsx')
matrix_agent_nophase = os.path.join('..', 'features', 'cleaned','matrix-2019-0411_15-30-16Features-agent-nophase.xlsx') """

# old dataset
"""matrix_subject_phases = os.path.join('..', 'features', 'matrix-2019-0226_15-16-44Features-157015.xlsx')
matrix_agent_phases = os.path.join('..', 'features', 'matrix-2019-0226_17-18-28Features-agent-157015.xlsx')
matrix_subject_nophase = os.path.join('..', 'features', 'matrix-2019-0227_20-11-53Features-nophase.xlsx')
matrix_agent_nophase = os.path.join('..', 'features', 'matrix-2019-0227_22-43-15Features-agent-nophase.xlsx') """

def prepare_df(df):
    # Replace 3 classes (low, medium, high) by 5 classes (ie, score rounded)
    #df['Presence Class'] = df['Presence Score'].round()
    #df['Co-presence Class'] = df['Co-presence Score'].round()
    #df = df.drop(['Presence Score', 'Co-presence Score'], axis=1)
    df = df.rename(index=str,
                   columns={"Presence Class": "PresenceClass", "Co-presence Class": "CopresenceClass"})
    return df

def prepare_agent_df(df):
    df = prepare_df(df)
    df = df.drop(['PresenceClass', 'CopresenceClass', 'Presence Score', 'Co-presence Score', 'Duration', 'Expert'], axis=1)
    agent_cols_rename = {}
    for col in df.columns:
        if col not in ['Candidate', 'Environment']:
            agent_cols_rename[col] = col + '_agent'
    df = df.rename(index=str, columns=agent_cols_rename)
    return df
    
def dfval(df, candidate, environment, column):
    return df.query("Candidate == '%s' & Environment == '%s'" % (candidate, environment))[column].values

print("Loading data - phases split ...")

subject_p_df = pd.read_excel(matrix_subject_phases).set_index('Candidate').drop('E6F', axis=0).reset_index()
subject_p_df = prepare_df(subject_p_df)
print("   Subject(phases): %d samples" % len(subject_p_df))
agent_p_df = pd.read_excel(matrix_agent_phases).set_index('Candidate').drop('E6F', axis=0).reset_index()
agent_p_df = prepare_agent_df(agent_p_df)
print("   Agent(phases): %d samples" % len(agent_p_df))
# add Duration feature from subject to agent - requires to first merge both datasets
agent_p_df = pd.merge(agent_p_df, subject_p_df[['Candidate', 'Environment', 'Duration', 'PresenceClass', 'CopresenceClass', 'Presence Score', 'Co-presence Score']], on=['Candidate', 'Environment'], 
                      how='inner', suffixes=(False, False))
print("   Agent(phases-merged): %d samples" % len(agent_p_df))

# no phase

subject_np_df = pd.read_excel(matrix_subject_nophase).set_index('Candidate').drop('E6F', axis=0).reset_index()
subject_np_df = prepare_df(subject_np_df)
print("   Subject(no phase): %d samples" % len(subject_np_df))
agent_np_df = pd.read_excel(matrix_agent_nophase).set_index('Candidate').drop('E6F', axis=0).reset_index()
agent_np_df = prepare_agent_df(agent_np_df)
print("   Agent(no phase): %d samples" % len(agent_p_df))
# add Duration feature from subject to agent - requires to first merge both datasets
agent_np_df = pd.merge(agent_np_df, subject_np_df[['Candidate', 'Environment', 'Duration', 'PresenceClass', 'CopresenceClass', 'Presence Score', 'Co-presence Score']], on=['Candidate', 'Environment'], 
                      how='inner', suffixes=(False, False))
print("   Agent(no phase-merged): %d samples" % len(agent_np_df))

subject_p_df.index = [subject_p_df['Candidate'], subject_p_df['Environment']]
agent_p_df.index = [agent_p_df['Candidate'], agent_p_df['Environment']]
subject_np_df.index = [subject_np_df['Candidate'], subject_np_df['Environment']]
agent_np_df.index = [agent_np_df['Candidate'], agent_np_df['Environment']]

intersect = set(subject_p_df.index) & set(agent_p_df.index) & set(subject_np_df.index) & set(agent_np_df.index)
print("  samples intersection %d" % len(intersect))

rejected_from_subject_p_df = set(subject_p_df.index) - intersect
rejected_from_subject_np_df = set(subject_np_df.index) - intersect
rejected_from_agent_p_df = set(agent_p_df.index) - intersect
rejected_from_agent_np_df = set(agent_np_df.index) - intersect

# restrict each dataset to the common samples
subject_p_df = subject_p_df.loc[list(intersect)].reset_index(drop=True)
subject_np_df = subject_np_df.loc[list(intersect)].reset_index(drop=True)
agent_p_df = agent_p_df.loc[list(intersect)].reset_index(drop=True)
agent_np_df = agent_np_df.loc[list(intersect)].reset_index(drop=True)

all_p_df = pd.merge(subject_p_df, agent_p_df.drop(['Duration', 'PresenceClass', 'CopresenceClass', 'Presence Score', 'Co-presence Score'], axis=1), on=['Candidate', 'Environment'], 
                    how='inner', suffixes=(False, False))
#all_p_df = all_p_df.drop(['PresenceClass_agent', 'CopresenceClass_agent', 'Duration_agent', 'Expert_agent'], axis=1)
print("   Merge(phases): %d samples " % len(all_p_df))

all_np_df = pd.merge(subject_np_df, agent_np_df.drop(['Duration', 'PresenceClass', 'CopresenceClass', 'Presence Score', 'Co-presence Score'], axis=1), on=['Candidate', 'Environment'], 
                     how='inner', suffixes=(False, False))
#all_np_df = all_np_df.drop(['PresenceClass_agent', 'CopresenceClass_agent', 'Duration_agent', 'Expert_agent'], axis=1)
print("   Merge(no phase): %d samples " % len(all_np_df))

print("")
print("Additional info:")
print("   samples rejected from subject / phases   %d" % len(rejected_from_subject_p_df))
for item in sorted(list(rejected_from_subject_p_df)) : print("     %s / %s" % (item[0], item[1]))
print("   samples rejected from subject / no phase %d" % len(rejected_from_subject_np_df))
for item in sorted(list(rejected_from_subject_np_df)) : print("     %s / %s" % (item[0], item[1]))
print("   samples rejected from agent / phases     %d" % len(rejected_from_agent_p_df))
for item in sorted(list(rejected_from_agent_p_df)) : print("     %s / %s" % (item[0], item[1]))
print("   samples rejected from agent / no phase   %d" % len(rejected_from_agent_np_df))
for item in sorted(list(rejected_from_agent_np_df)) : print("     %s / %s" % (item[0], item[1]))
    
# compute new features being mean of 'yaw', 'pitch' and 'roll'

for feature in ['%s%sAngularSpeed_Head_%s', 'Avg_%s%sAngularSpeed_Hand_%s']:
    for stat in ['Mean', 'Std']:
        for phase in ['Start', 'Mid', 'End']:
            if phase == 'Mid':
                all_np_df[feature % (stat, 'Mean', phase)] = all_np_df[[feature % (stat, 'Yaw', phase), 
                                                                        feature % (stat, 'Pitch', phase), 
                                                                        feature % (stat, 'Roll', phase)]].mean(axis=1)
                all_np_df[feature % (stat, 'Mean', phase)+'_agent'] = all_np_df[[feature % (stat, 'Yaw', phase)+'_agent', 
                                                                        feature % (stat, 'Pitch', phase)+'_agent', 
                                                                        feature % (stat, 'Roll', phase)+'_agent']].mean(axis=1)        
                subject_np_df[feature % (stat, 'Mean', phase)] = subject_np_df[[feature % (stat, 'Yaw', phase), 
                                                                        feature % (stat, 'Pitch', phase), 
                                                                        feature % (stat, 'Roll', phase)]].mean(axis=1)
                agent_np_df[feature % (stat, 'Mean', phase)+'_agent'] = agent_np_df[[feature % (stat, 'Yaw', phase)+'_agent', 
                                                                        feature % (stat, 'Pitch', phase)+'_agent', 
                                                                        feature % (stat, 'Roll', phase)+'_agent']].mean(axis=1)
            all_p_df[feature % (stat, 'Mean', phase)] = all_p_df[[feature % (stat, 'Yaw', phase), 
                                                                    feature % (stat, 'Pitch', phase), 
                                                                    feature % (stat, 'Roll', phase)]].mean(axis=1)
            all_p_df[feature % (stat, 'Mean', phase)+'_agent'] = all_p_df[[feature % (stat, 'Yaw', phase)+'_agent', 
                                                                    feature % (stat, 'Pitch', phase)+'_agent', 
                                                                    feature % (stat, 'Roll', phase)+'_agent']].mean(axis=1)        
            subject_p_df[feature % (stat, 'Mean', phase)] = subject_p_df[[feature % (stat, 'Yaw', phase), 
                                                                    feature % (stat, 'Pitch', phase), 
                                                                    feature % (stat, 'Roll', phase)]].mean(axis=1)
            agent_p_df[feature % (stat, 'Mean', phase)+'_agent'] = agent_p_df[[feature % (stat, 'Yaw', phase)+'_agent', 
                                                                    feature % (stat, 'Pitch', phase)+'_agent', 
                                                                    feature % (stat, 'Roll', phase)+'_agent']].mean(axis=1)

# drop useless columns
all_np_df.drop([col for col in all_np_df.columns if 'Yaw' in col or 'Roll' in col or 'Pitch' in col or 'Freq_' in col], axis=1, inplace=True)
all_p_df.drop([col for col in all_p_df.columns if 'Yaw' in col or 'Roll' in col or 'Pitch' in col or 'Freq_' in col], axis=1, inplace=True)
subject_np_df.drop([col for col in subject_np_df.columns if 'Yaw' in col or 'Roll' in col or 'Pitch' in col or 'Freq_' in col], axis=1, inplace=True)
subject_p_df.drop([col for col in subject_p_df.columns if 'Yaw' in col or 'Roll' in col or 'Pitch' in col or 'Freq_' in col], axis=1, inplace=True)
agent_np_df.drop([col for col in agent_np_df.columns if 'Yaw' in col or 'Roll' in col or 'Pitch' in col or 'Freq_' in col], axis=1, inplace=True)
agent_p_df.drop([col for col in agent_p_df.columns if 'Yaw' in col or 'Roll' in col or 'Pitch' in col or 'Freq_' in col], axis=1, inplace=True)

# handle categorical data type
all_np_df['Expert'] = all_np_df['Expert'].astype('category')
all_p_df['Expert'] = all_p_df['Expert'].astype('category')
subject_np_df['Expert'] = subject_np_df['Expert'].astype('category')
subject_p_df['Expert'] = subject_p_df['Expert'].astype('category')


## Presence and co-presence

In [ ]:
plt.hist(all_p_df['Presence Score'], bins=20, range=(1,5), edgecolor='black')
plt.ylabel("Number of samples")
plt.xlabel("Presence score")
plt.axvline(x=all_p_df['Presence Score'].mean(), c='red', label='mean (%.02f)'%all_p_df['Presence Score'].mean())
plt.axvline(x=all_p_df['Presence Score'].median(), c='green', label='median (%.02f)'%all_p_df['Presence Score'].median())
plt.legend()
plt.show()
plt.hist(all_p_df['Co-presence Score'], bins=20, range=(1,5), edgecolor='black')
plt.axvline(x=all_p_df['Co-presence Score'].mean(), c='red', label='mean (%.02f)'%all_p_df['Co-presence Score'].mean())
plt.axvline(x=all_p_df['Co-presence Score'].median(), c='green', label='median (%.02f)'%all_p_df['Co-presence Score'].median())
plt.legend()
plt.ylabel("Number of samples")
plt.xlabel("Co-Presence score")
plt.show()

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn import metrics


discretization_params = {
    'n_bins' : [2,3,4,5,6,7,8,9,10],
    'strategy' : ['uniform', 'quantile', 'kmeans']
}

def discretize(data, n_bins, strategy):
    enc = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy=strategy)
    pres_arr = np.array(data).reshape(-1,1)
    pres_kmeans = enc.fit_transform(pres_arr).astype(int)
    return pres_kmeans, enc

def compute_discretization_scores(data, labels):
    calinski = metrics.calinski_harabaz_score(data, labels)
    silhouette = metrics.silhouette_score(data, labels, metric='euclidean')
    db = metrics.davies_bouldin_score(data, labels)
    min_cluster = np.unique(labels, return_counts=True)[1].min()
    max_cluster = np.unique(labels, return_counts=True)[1].max()  
    return [calinski, silhouette, db, min_cluster, max_cluster]

def discretization_gridsearch(data, discretization_params, orig_discretization=None, title_prefix='presence, '):
    data = np.array(data).reshape(-1, 1)
    discretization_params_grid = ParameterGrid(discretization_params)
    results = np.zeros(shape=(len(discretization_params_grid)+1, 5))
    for idx, param_set in enumerate(discretization_params_grid):
        print(idx, param_set)
        pres_kmeans, enc = discretize(data, n_bins=param_set['n_bins'], strategy=param_set['strategy'])
        bins = np.bincount(pres_kmeans.astype(int).flatten())
        plt.bar(range(len(bins)), bins, color='grey')
        #plt.hist(pres_kmeans.astype(int))
        plt.title('%s%d clusters, strategy %s, bins edges %s' % (title_prefix, param_set['n_bins'], param_set['strategy'], enc.bin_edges_))
        plt.show()

        results[idx] = compute_discretization_scores(data, pres_kmeans)
    if orig_discretization is not None:
        results[-1] = compute_discretization_scores(data, orig_discretization)
    myindex = ["%s (%s)" % (p['n_bins'], p['strategy']) for p in discretization_params_grid]
    myindex.append("3 (orig)")
    cluster_results = pd.DataFrame(data=results, index=myindex, columns=['Calinski-Harabaz', 'Silhouette', 'Davies-Bouldin',
                                                                        'min cluster nb', 'max cluster nb'])
    cluster_results['C-H'] = (cluster_results['Calinski-Harabaz'] - cluster_results['Calinski-Harabaz'].mean()) / cluster_results['Calinski-Harabaz'].std()
    cluster_results['S'] = (cluster_results['Silhouette'] - cluster_results['Silhouette'].mean()) / cluster_results['Silhouette'].std()
    cluster_results['D-B'] = (cluster_results['Davies-Bouldin'] - cluster_results['Davies-Bouldin'].mean()) / cluster_results['Davies-Bouldin'].std()
    cluster_results['Weighted'] = cluster_results['D-B'] - cluster_results['C-H'] - cluster_results['S']
    return cluster_results

def highlight_max(x):
    return ['color: red' if v == x.max() else 'color: green' if v == x.min() else '' for v in x]

presence = all_p_df.sort_values(by=['Presence Score'])['Presence Score']
copresence = all_p_df.sort_values(by=['Co-presence Score'])['Co-presence Score']
orig_pres = all_p_df.sort_values(by=['Presence Score'])['PresenceClass']
orig_copres = all_p_df.sort_values(by=['Co-presence Score'])['CopresenceClass']
print(len(presence))
print(len(orig_pres))

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
pres_clusters_results = discretization_gridsearch(presence, discretization_params, orig_pres, 'presence, ')
copres_clusters_results = discretization_gridsearch(copresence, discretization_params, orig_copres, 'co-presence, ')
np.set_printoptions()

In [ ]:
#pres_clusters_results.style.apply(highlight_max).to_excel(os.path.join(config.OUT_PATH, 'presence_discretization.xlsx'))
#copres_clusters_results.style.apply(highlight_max).to_excel(os.path.join(config.OUT_PATH, 'copresence_discretization.xlsx'))

def highlight_low_clusters(s):
    if s['min cluster nb'] < 7.0: # because upsampling algorithms consider 6 neighbors
        return ['color: red']*9
    else:
        return ['background-color: white']*9

pres_clusters_results.sort_values(by=['Weighted']).style.apply(highlight_low_clusters, axis=1).to_excel(os.path.join(config.OUT_PATH, 'presence_discretization.xlsx'))
copres_clusters_results.sort_values(by=['Weighted']).style.apply(highlight_low_clusters, axis=1).to_excel(os.path.join(config.OUT_PATH, 'copresence_discretization.xlsx'))
pres_clusters_results.sort_values(by=['Weighted']).style.apply(highlight_low_clusters, axis=1)

In [ ]:
copres_clusters_results.sort_values(by=['Weighted']).style.apply(highlight_low_clusters, axis=1)

Best discretization corresponds to highest values for Calinsky-Harabaz and Silouhette, and lowest values for Davies-Bouldin indices.
Columns C-H, S and D-B are centered / reduced values of each index, weighted is D-B minus S and C-H. By looking at lowest values in 'Weighted' we hope to maximize C-H and S while minimizing D-B.
(lines in red are suppressed because they have too small cluster(s)).

But effect on learning tasks has to be experimented. Results can be confirmed as interesting based on table above.

## Features definition
### Features

In [ ]:
# features including phases

feats_df_index = ['type']
feats_df_columns = all_p_df.columns

feats_df = pd.DataFrame(index=feats_df_index, columns=feats_df_columns)
feats_df.head()

### Features sets

In [ ]:
# features including phases

features_df_index = [
             ['Doctor', 'Agent', 'Doctor+Agent'], 
             ['No Phase', '157015', 'Verbal', 'Non-Verbal', 'Multimodal', 'Non-Verbal-A', 'Multimodal-A', 
              'No Phase-A', '157015-A', 'No Phase-FULL', '157015-FULL']   #, 'Multimodal+Duration']
            ]
features_df_columns = ['precision', 'f1', 'recall', 'params']
features_df_multiindex = pd.MultiIndex.from_product(features_df_index, 
                                                  names=['Subject', 'Feature Set'])

features_df = pd.DataFrame(index=features_df_multiindex)
features_df.head()

In [ ]:
def get_features_set(subject_name, set_name):
    row = features_df.loc[(subject_name, set_name)]
    return features_df.columns[row == True]

# Load features sets definition
features_df = pd.read_excel(os.path.join(config.OUT_PATH, 'features_sets.xlsx'))
features_df.index = features_df_multiindex
features_df = features_df.drop(['Subject', 'Feature Set'], axis=1)
features_df.head()

In [ ]:
### Optional - (re)initialize features sets if necessary

mean_angular_features = [
    "MeanMeanAngularSpeed_Head_Start", "MeanMeanAngularSpeed_Head_Mid", "MeanMeanAngularSpeed_Head_End",
    "Avg_MeanMeanAngularSpeed_Hand_Start", "Avg_MeanMeanAngularSpeed_Hand_Mid", "Avg_MeanMeanAngularSpeed_Hand_End"
]
std_angular_features = [
    "StdMeanAngularSpeed_Head_Start", "StdMeanAngularSpeed_Head_Mid", "StdMeanAngularSpeed_Head_End",                    
    "Avg_StdMeanAngularSpeed_Hand_Start", "Avg_StdMeanAngularSpeed_Hand_Mid", "Avg_StdMeanAngularSpeed_Hand_End"
]
subject_features = [
    'Expert', "Head_Entropy_Start", "Head_Entropy_Mid", "Head_Entropy_End", "Avg_HandEntropy_Begin",
    "Avg_HandEntropy_Mid", "Avg_HandEntropy_End", "Avg_SentenceLength_Begin", "Avg_SentenceLength_Mid",
    "Avg_SentenceLength_End",  "Avg_AnswersDelay_Begin", "Avg_AnswersDelay_Mid", "Avg_AnswersDelay_End",
    "Avg_IPUlen_Begin", "Avg_IPUlen_Mid", "Avg_IPUlen_End",
    "Ratio1_Begin", "Ratio1_Mid", "Ratio1_End", "Ratio2_Begin", "Ratio2_Mid", 
    "Ratio2_End", "Duration"] 

agent_features = [feat+'_agent' for feat in subject_features if feat not in ['Expert', 'Duration'] and 'Angular' not in feat] + ['Duration']
# not using "Expert" feature for agent features
#agent_features = subject_features - {"Expert"}
all_features = subject_features + agent_features[:-1]

subject_features_a = [feat for feat in subject_features if not 'Entropy' in feat] + mean_angular_features + std_angular_features
agent_features_a = [feat+'_agent' for feat in subject_features_a if feat not in ['Expert', 'Duration']] + ['Duration']
# not using "Expert" feature for agent features
#agent_features = subject_features - {"Expert"}
all_features_a = subject_features_a + agent_features_a[:-1]

subject_features_full = subject_features + mean_angular_features + std_angular_features
all_features_full = subject_features_full + agent_features_a[:-1]

features_df = pd.DataFrame(index=features_df_multiindex, columns=sorted(list(set(all_features+all_features_a))))


# features - no phase

subject_features_nophase = [feat for feat in subject_features if not (feat.endswith('_Begin') 
                            or feat.endswith('_End') or feat.endswith('_Start') or 'Angular' in feat)]
#["Expert", "Head_Entropy_Mid", "Avg_HandEntropy_Mid", "Avg_SentenceLength_Mid", "Ratio1_Mid", 
#                            "Ratio2_Mid", "Duration"]
agent_features_nophase = [feat for feat in agent_features if not (feat.endswith('_Begin_agent') 
                            or feat.endswith('_End_agent') or feat.endswith('_Start_agent') or 'Angular' in feat)]
print(agent_features)
print(agent_features_nophase)
all_features_nophase = subject_features_nophase + agent_features_nophase[:-1]


subject_features_nophase_a = [feat for feat in subject_features_a if not (feat.endswith('_Begin') 
                            or feat.endswith('_End') or feat.endswith('_Start'))]
#["Expert", "Head_Entropy_Mid", "Avg_HandEntropy_Mid", "Avg_SentenceLength_Mid", "Ratio1_Mid", 
#                            "Ratio2_Mid", "Duration"]
agent_features_nophase_a = [feat for feat in agent_features_a if not (feat.endswith('_Begin_agent') 
                            or feat.endswith('_End_agent') or feat.endswith('_Start_agent'))]
print(agent_features)
print(agent_features_nophase)
all_features_nophase_a = subject_features_nophase_a + agent_features_nophase_a[:-1]

subject_features_nophase_full =  [feat for feat in subject_features_full if not (feat.endswith('_Begin') 
                            or feat.endswith('_End') or feat.endswith('_Start'))]
all_features_nophase_full = subject_features_nophase_full + agent_features_nophase_a[:-1]             

# for modalities verbal, non-verbal etc, we do not consider phases
# verbal features

subject_features_verbal = ['Avg_AnswersDelay_Mid', "Avg_SentenceLength_Mid", "Avg_IPUlen_Mid", "Ratio1_Mid", "Ratio2_Mid"]
agent_features_verbal = [feat+'_agent' for feat in subject_features_verbal]
all_features_verbal = subject_features_verbal + agent_features_verbal

# non-verbal features

subject_features_nonverbal = ["Head_Entropy_Mid", "Avg_HandEntropy_Mid"]
agent_features_nonverbal = [feat+'_agent' for feat in subject_features_nonverbal]
all_features_nonverbal = subject_features_nonverbal + agent_features_nonverbal

subject_features_nonverbal_a =  [f for f in mean_angular_features if '_Mid' in f and ('_Head' in f or 'Avg_' in f)] + [f for f in std_angular_features if '_Mid' in f and ('_Head' in f or 'Avg_' in f)]
agent_features_nonverbal_a = [feat+'_agent' for feat in subject_features_nonverbal_a]
all_features_nonverbal_a = subject_features_nonverbal_a + agent_features_nonverbal_a

# multimodal features

subject_features_multimodal = subject_features_verbal + subject_features_nonverbal
agent_features_multimodal = [feat+'_agent' for feat in subject_features_multimodal]
all_features_multimodal = subject_features_multimodal + agent_features_multimodal

subject_features_multimodal_a = subject_features_verbal + subject_features_nonverbal_a
agent_features_multimodal_a = [feat+'_agent' for feat in subject_features_multimodal_a]
all_features_multimodal_a = subject_features_multimodal_a + agent_features_multimodal_a

# multimodal + duration features

#subject_features_multimodal_duration = subject_features_multimodal + ['Duration']
#agent_features_multimodal_duration = agent_features_multimodal + ['Duration']
#all_features_multimodal_duration = subject_features_multimodal_duration + agent_features_multimodal

for feat in features_df.columns:
    features_df.loc[('Doctor', 'No Phase'), feat] = True if feat in subject_features_nophase else False
    features_df.loc[('Doctor', '157015'), feat] = True if feat in subject_features else False
    features_df.loc[('Doctor', 'No Phase-A'), feat] = True if feat in subject_features_nophase_a else False
    features_df.loc[('Doctor', '157015-A'), feat] = True if feat in subject_features_a else False    
    features_df.loc[('Doctor', 'No Phase-FULL'), feat] = True if feat in subject_features_nophase_full else False
    features_df.loc[('Doctor', '157015-FULL'), feat] = True if feat in subject_features_full else False    
    features_df.loc[('Doctor', 'Verbal'), feat] = True if feat in subject_features_verbal else False
    features_df.loc[('Doctor', 'Non-Verbal'), feat] = True if feat in subject_features_nonverbal else False
    features_df.loc[('Doctor', 'Non-Verbal-A'), feat] = True if feat in subject_features_nonverbal_a else False    
    features_df.loc[('Doctor', 'Multimodal'), feat] = True if feat in subject_features_multimodal else False
    features_df.loc[('Doctor', 'Multimodal-A'), feat] = True if feat in subject_features_multimodal_a else False    
    #features_df.loc[('Doctor', 'Multimodal+Duration'), feat] = True if feat in subject_features_multimodal_duration else False
    features_df.loc[('Agent', 'No Phase'), feat] = True if feat in agent_features_nophase else False
    features_df.loc[('Agent', '157015'), feat] = True if feat in agent_features else False
    features_df.loc[('Agent', 'No Phase-A'), feat] = True if feat in agent_features_nophase_a else False
    features_df.loc[('Agent', '157015-A'), feat] = True if feat in agent_features_a else False    
    features_df.loc[('Agent', 'No Phase-FULL'), feat] = True if feat in agent_features_nophase_a else False
    features_df.loc[('Agent', '157015-FULL'), feat] = True if feat in agent_features_a else False    
    features_df.loc[('Agent', 'Verbal'), feat] = True if feat in agent_features_verbal else False
    features_df.loc[('Agent', 'Non-Verbal'), feat] = True if feat in agent_features_nonverbal else False
    features_df.loc[('Agent', 'Non-Verbal-A'), feat] = True if feat in agent_features_nonverbal_a else False    
    features_df.loc[('Agent', 'Multimodal'), feat] = True if feat in agent_features_multimodal else False
    features_df.loc[('Agent', 'Multimodal-A'), feat] = True if feat in agent_features_multimodal_a else False
    #features_df.loc[('Agent', 'Multimodal+Duration'), feat] = True if feat in agent_features_multimodal_duration else False
    features_df.loc[('Doctor+Agent', 'No Phase'), feat] = True if feat in all_features_nophase else False
    features_df.loc[('Doctor+Agent', '157015'), feat] = True if feat in all_features else False
    features_df.loc[('Doctor+Agent', 'No Phase-A'), feat] = True if feat in all_features_nophase_a else False
    features_df.loc[('Doctor+Agent', '157015-A'), feat] = True if feat in all_features_a else False
    features_df.loc[('Doctor+Agent', 'No Phase-FULL'), feat] = True if feat in all_features_nophase_full else False
    features_df.loc[('Doctor+Agent', '157015-FULL'), feat] = True if feat in all_features_full else False
    features_df.loc[('Doctor+Agent', 'Verbal'), feat] = True if feat in all_features_verbal else False
    features_df.loc[('Doctor+Agent', 'Non-Verbal'), feat] = True if feat in all_features_nonverbal else False
    features_df.loc[('Doctor+Agent', 'Non-Verbal-A'), feat] = True if feat in all_features_nonverbal_a else False    
    features_df.loc[('Doctor+Agent', 'Multimodal'), feat] = True if feat in all_features_multimodal else False
    features_df.loc[('Doctor+Agent', 'Multimodal-A'), feat] = True if feat in all_features_multimodal_a else False
    #features_df.loc[('Doctor+Agent', 'Multimodal+Duration'), feat] = True if feat in all_features_multimodal_duration else False

features_df.to_excel(os.path.join(config.OUT_PATH, 'features_sets.xlsx'))

In [ ]:
import re
from functools import cmp_to_key

def cmp_feats(a,b):
    #print('%s - %s' % (a,b))
    
    if a == b:
        return 0
    
    r = r"(.*)(_Begin|_Start|_Mid|_End)(_agent)?"
    ar = re.findall(r, a)
    if len(ar) > 0:
        ar = [val for val in ar[0] if val is not '']
    else:
        ar = [a]
    br = re.findall(r, b)
    if len(br) > 0:
        br = [val for val in br[0] if val is not '']
    else:
        br = [b]
    #print(str(ar) + ' ' + str(len(ar)))
    #print(str(br) + ' ' + str(len(br)))
    
    if len(ar) == 1:
        if len(br) == 1:
            if a > b:
                return 1
            else:
                return -1
        else:
            return -1
    if len(br) == 1:
        return 1
    
    if len(ar) == 2:
        if len(br) == 2:
            if ar[0] != br[0]:
                if a > b:
                    return 1
                else:
                    return -1
            if ar[1] == '_Begin' or ar[1] == '_Start':
                return -1
            if br[1] == '_Begin' or br[1] == '_Start':
                return 1
            if ar[1] == '_End':
                return 1
            if br[1] == '_End':
                return -1
            
    if len(ar) == 3:
        if len(br) == 2:
            return 1
        #print('AGENT')
        return cmp_feats(ar[0]+ar[1], br[0]+br[1])
        
    if len(br) == 3:
        return -1
    

l = list(features_df.columns)
#print(l)
l = sorted(l, key=cmp_to_key(cmp_feats))
features_df = features_df[l]
#print(features_df.columns)

plt.figure(figsize=(20,10))
sns.heatmap(features_df.fillna(0).astype('int'), linecolor='white', linewidth=1, annot=False, cbar=False)
plt.savefig(os.path.join(config.OUT_PATH, 'features_sets.png'), bbox_inches = "tight")
plt.show()


Export datasets as Excel sheets for other analysis.

## Dataset preparation

In [ ]:
export_df = False

if export_df:
    cols = list(get_features_set('Doctor+Agent', 'No Phase-FULL'))
    cols.append('Candidate')
    cols.append('Environment')
    cols.append('Presence Score')
    cols.append('Co-presence Score')
    print(all_np_df.columns)
    df = all_np_df[cols]
    df = df.dropna(axis='columns', how='all')
    df = df.loc[:, (df != 0).any(axis=0)]
    l = list(df.columns)
    print(l)
    l.sort(cmp_feats)
    df = df[l]
    display(HTML(df.to_html()))
    df.to_excel('dataset_nophase.xlsx')
    
    cols = list(get_features_set('Doctor+Agent', '157015-FULL'))
    cols.append('Candidate')
    cols.append('Environment')
    cols.append('Presence Score')
    cols.append('Co-presence Score')
    print(all_p_df.columns)
    df = all_p_df[cols]
    df = df.dropna(axis='columns', how='all')
    df = df.loc[:, (df != 0).any(axis=0)]
    l = list(df.columns)
    print(l)
    l.sort(cmp_feats)
    df = df[l]
    display(HTML(df.to_html()))
    df.to_excel('dataset_phases.xlsx')

In [ ]:
def get_clf_from_estimator(estimator):
    """
    Returns the classifier from a built estimator (that could be a classifier already, or a pipeline including a classifier).
    If there is none returns None.
    """
    if not isinstance(estimator, Pipeline):
        return estimator
    else:
        if 'clf' in estimator.named_steps:
            return estimator.named_steps['clf']
    
    return None
           

def discretize_class(samples, prediction_type='classification', prediction_task='presence', bins=None, strategy=None, test_set=0.1, verbose=0):
    """
    Discretizes presence or co-presence score and put result in a new column (if not already existing).
    
    Returns: the name of the new column, or None if one of discretized groups only has 2 samples or less, or if discretization
    failed for any reason.
    """
    
    if prediction_type == 'regression':
        if prediction_task == 'presence':
            return 'Presence Score'
        else:
            return 'Co-presence Score'
    
    cols = []
    for col in samples.columns:
        if 'core' in col:
            cols.append(col)
    if verbose > 1:
        print('discretize_class columns %s' % str(cols))
    if bins is None or strategy is None:
        if prediction_task == 'presence':
            class_column='PresenceClass'
        else:
            class_column='CopresenceClass'
        return class_column
    else:
        if prediction_task == 'presence':
            score_column = 'Presence Score'
            class_column = 'PresenceClass_%d_%s' % (bins, strategy)
        else:
            score_column = 'Co-presence Score'
            class_column = 'CopresenceClass_%d_%s' % (bins, strategy)      
    
    if class_column not in samples.columns:
        samples[class_column], enc = discretize(samples[score_column], bins, strategy)
        samples[class_column] = samples[class_column].astype(int)
        #TODO: transform test data with discretizer fit on train data
    
    if any(samples[class_column] == -1) or any(samples[class_column].value_counts() < 3):
        return None
    else:
        return class_column

def prepare_train_data(samples, features, prediction_task, bins=None, strategy=None, upsample=None, 
                       test_set=0.1, n_splits=1, splits_mode='shuffle', verbose=0):
    """
    Prepares a dataset for ML.
    
    Parameters
    ----------
    
    samples: DataFrame
    
    features: list
    
    prediction_task: str
        'presence' or 'copresence'
        
    bins: None or int, default None.
        Number of classes to create when discretizing presence and co-presence.
        If 0 there no discretization is applied and returned y is for a regression task.
    
    strategy: None, 'kmeans', 'uniform' or 'quantile', default None.
        See sklearn, if None defaults to original strategy.
        
    upsample: None or str, default None
        None: no oversampling
        For description of other oversampling method see https://imbalanced-learn.readthedocs.io/en/stable/over_sampling.html
        imb_random
        imb_smote
        imb_adasyn
        imb_smotenc
        imb_borderlinesmote1
        imb_borderlinesmote2
        imb_svmsmote
        
    test_set: None or float, default 0.1
        If not None or 0, a test set will produced with specified float as test ratio
        
    n_splits: int, default 1
        Will produce this number of train/test splits.
        
    splits_mode: str, default 'shuffle'
        'shuffle' to produce n_splits stratified and shuffled splits of the data.
        'kfold' to produce n_splits repetitions of K-folds splits, K being set to approximately respect test_set ratio.
        
    Returns
    -------
    An array of tuples (X, y, X_test, y_test).
    
    """
    if verbose > 1:
        print('prepare_train_data(samples nb=%d, features=%s, prediction_task=%s, bins=%s, strategy=%s, upsample=%s, test_set=%f, n_splits=%d)' 
              % (len(samples), str(features), prediction_task, str(bins), str(strategy), str(upsample), test_set, n_splits))
        #print('prepare_train_data: unique %d' %  len(np.unique(samples.loc[:, features], return_counts=True, axis=0)[1]))
    
    return_tuples = []
    
    # TODO: should fit discretizer on train then transform train and test
    if bins == 0:
        class_column = 'Presence Score'
    else:
        class_column = discretize_class(samples, 'classification', prediction_task, bins, strategy, verbose)
    if verbose > 1:
        print('class_column: %s' % class_column)
    if test_set is not None and test_set > 0:
        if bins == 0:
            # regression
            sss = ShuffleSplit(n_splits=n_splits, test_size=test_set, random_state=RANDOM_STATE)
            splits = [[split[0], split[1]] for split in sss.split(np.zeros(len(samples)), samples[class_column])]
        else:
            if splits_mode == 'shuffle':
                sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_set, random_state=RANDOM_STATE)
                splits = [[split[0], split[1]] for split in sss.split(np.zeros(len(samples)), samples[class_column])]
            else:
                nb_folds = int(1/test_set)
                sss = RepeatedStratifiedKFold(n_splits=nb_folds, n_repeats=n_splits, random_state=RANDOM_STATE)
                splits = [[split[0], split[1]] for split in sss.split(np.zeros(len(samples)), samples[class_column])]
    else:
        splits = [[None, None]]
    if verbose > 1:
        print("splits " + str(splits))
    for idx, split in enumerate(splits):
        train_idx = split[0]
        test_idx = split[1]
        if verbose > 1:
            print("train " + str(train_idx) + ", test " + str(test_idx))
        if train_idx is None or test_idx is None:
            X = samples.loc[:, features].fillna(0)
            y = samples[class_column]
            X_test = None
            y_test = None
        else:
            X = samples.loc[train_idx, features].fillna(0)
            X_test = samples.loc[test_idx, features].fillna(0)
            y = samples.loc[train_idx, class_column]
            y_test = samples.loc[test_idx, class_column]        
        
        ros = None
        if upsample == 'imb_random':
            ros = RandomOverSampler(random_state=RANDOM_STATE)
        elif upsample == 'imb_smote':
            ros = SMOTE(random_state=RANDOM_STATE) 
        elif upsample == 'imb_adasyn':
            ros = ADASYN(random_state=RANDOM_STATE)
        elif upsample == 'imb_smotenc':
            if 'Expert' in features:        
                # SMOTENC is the only one dealing specifically with categorical data (ie 'Expert' feature)
                categorical_vars_indices = [list(features).index('Expert')]
                ros = SMOTENC(random_state=RANDOM_STATE, categorical_features=categorical_vars_indices)
            else:
                ros = SMOTE(random_state=RANDOM_STATE)

        elif upsample == 'imb_borderlinesmote1':
            ros = BorderlineSMOTE(kind='borderline-1', random_state=RANDOM_STATE)      
        elif upsample == 'imb_borderlinesmote2':
            ros = BorderlineSMOTE(kind='borderline-2', random_state=RANDOM_STATE)     
        elif upsample == 'imb_svmsmote':
            ros = SVMSMOTE(random_state=RANDOM_STATE)

        if ros is not None:
            # test set is never oversampled
            if verbose > 1:
                print("prepare_train_data: Applying %s oversampling to data %d / %s / unique %d" 
                      % (ros.__class__.__name__, len(X), collections.Counter(y), 
                         len(X.drop_duplicates()) )) 
            X, y = ros.fit_resample(X, y)     
            X = pd.DataFrame(X, columns=features)
            if verbose > 1:
                print("prepare_train_data: Resampled data %d / %s / unique %d" 
                      % (len(X), collections.Counter(y), len(X.drop_duplicates())))

        # ! not needed anymore because .arff needs to be generated on-the-fly (to take into consideriation
        # a pipeline where dataset could be standard scaled, features selected, etc)
        # special treatment for JNCC2, generate .arff files
        """if isinstance(clf, JNCC2Wrapper) and False:
            if bins == 0:
                print("ERROR: arff can only be generated for classification task")
                return None
            else:
                arff_path = os.path.join(clf.arff_root_path_, str(idx))
                clf.generate_arff(os.path.join(arff_path, 'train'), features, prediction_task, X, y)
                if test_set is not None and test_set > 0:
                    clf.generate_arff(os.path.join(arff_path, 'test'), features, prediction_task, X_test, y_test)
        """
        
                
        return_tuples.append( (X, y, X_test, y_test) )
        
    return return_tuples

def build_scoring(clf, y):
    labels = sorted(np.unique(y).astype(int))
    
    scoring = {}
    scoring['f1_macro_weighted'] = 'f1_weighted'
    for idx, label in enumerate(labels):
        scoring['f1_macro_class_%d' % label] = make_scorer(f1_score, labels=[label], average='weighted')
    scoring['precision_macro_weighted'] = 'precision_weighted'
    for idx, label in enumerate(labels):
        scoring['precision_macro_class_%d' % label] = make_scorer(precision_score, labels=[label], average='weighted')    
    scoring['recall_macro_weighted'] = 'recall_weighted'
    for idx, label in enumerate(labels):
        scoring['recall_macro_class_%d' % label] = make_scorer(recall_score, labels=[label], average='weighted')     
    #scoring['f1_micro'] = 'f1_micro'
    
    if type(clf) == RandomForestClassifier:
        scoring ['oob'] = score_forest
    else:
        scoring['fi'] = svm_fi    
    if isinstance(clf, JNCC2Wrapper) or (isinstance(clf, Pipeline) and 'clf' in clf.named_steps 
                                        and isinstance(clf.named_steps['clf'], JNCC2Wrapper)):
        scoring['mrr'] = make_scorer(lambda yt,yp: feu.compute_mrr(yt, yp, labels))
        
        # Note: determinacy is useless for cross-validation scores, as in this case JNCC2 will return only deterministic
        # predictions - this score would always be 1. It then makes sense only for test scores (where JNCC2 may return
        #  non-deterministic predictions)
        #scoring['determinacy'] = make_scorer(lambda yt, yp: np.sum(np.sum(yp != 6666, axis=1) == 1) / float(len(yp)))

    return scoring

def gridsearch(clf, X, y, prediction_task, param_grid, features, cv=10, n_jobs=-1, verbose=0):
    if verbose > 1:
        print("gridsearch(clf=%s, modelTarget=%s, param_grid=%s, features=%s)" 
              % (type(clf), prediction_task, param_grid, features))

    #n_estimators = np.concatenate((np.arange(1,10), np.arange(10,100,10)))
    print(np.unique(y, return_counts=True))
    grid = GridSearchCV(estimator=clf, param_grid=param_grid,
                        scoring=['f1_weighted', 'precision_weighted', 'recall_weighted'],
                        refit='f1_weighted',
                        cv=cv,
                        return_train_score=True,
                        n_jobs=n_jobs if not isinstance(get_clf_from_estimator(clf), JNCC2Wrapper) else 1,
                        verbose=max(0, verbose-1))
    grid = grid.fit(X, y)

    results = grid.cv_results_
    # print("best params ", grid.best_params_)
    # print("best score ", grid.best_score_)

    return grid

def gridsearch2(clf, X, y, prediction_task, param_grid, features, cv=10, n_jobs=-1, verbose=0):
    if verbose > 1:
        print("gridsearch(clf=%s/%s, prediction_task=%s, param_grid=%s, features=%s, cv=%d, n_jobs=%d)" 
              % (type(clf), type(get_clf_from_estimator(clf)), prediction_task, param_grid, features, cv, n_jobs))

    if verbose > 2:
        print('gridsearch: support %s' % str(np.unique(y, return_counts=True)))
    grid = GridSearchCV(estimator=clf, param_grid=param_grid,
                        scoring=build_scoring(clf, y),
                        refit='f1_macro_weighted',
                        cv=cv,
                        return_train_score=True,
                        n_jobs=n_jobs,
                        verbose=max(0, verbose-1))
    grid = grid.fit(X, y)

    return grid



## Feature importance

In [ ]:
# from rfpimp

def dropcol_importances_sklearn(model, X_train, y_train, X_valid=None, y_valid=None, metric=None, sample_weights=None):
    """
    Compute drop-column feature importances for scikit-learn.

    Given a classifier or regression in model
    and training X and y data, return a data frame with columns
    Feature and Importance sorted in reverse order by importance.

    A clone of model is trained once to get the baseline score and then
    again, once per feature to compute the drop in either the model's .score() output
    or a custom metric callable in the form of metric(model, X_valid, y_valid).
    In case of a custom metric the X_valid and y_valid parameters should be set.
    return: A data frame with Feature, Importance columns

    SAMPLE CODE
    rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
    X_train, y_train = ..., ...
    rf.fit(X_train, y_train)
    imp = dropcol_importances(rf, X_train, y_train)
    """

    if X_valid is None: X_valid = X_train
    if y_valid is None: y_valid = y_train
    model_ = clone(model)
    #model_.random_state = RANDOM_STATE
    model_.fit(X_train, y_train)

    if callable(metric):
        baseline = metric(model_, X_valid, y_valid)
    else:
        baseline = model_.score(X_valid, y_valid)
    print("baseline " + str(baseline))
    imp = []
    for col in X_train.columns:
        print("column " + col)
        model__ = clone(model)
        #model__.random_state = RANDOM_STATE
        X_tr = X_train.drop(col, axis=1)
        X_va = X_valid.drop(col, axis=1)
        model__.fit(X_tr, y_train)
        if callable(metric):
            s = metric(model__, X_va, y_valid)
        else:
            s = model__.score(X_va, y_valid)
        print("new score " + str(s))
        drop_in_score = baseline - s
        imp.append(drop_in_score)

    imp = np.array(imp)

    #I = pd.DataFrame(data={'Feature':X_train.columns, 'Importance':imp})
    #I = I.set_index('Feature')
    #I = I.sort_values('Importance', ascending=False)
    
    return imp #I

def dropcol_importances(rf, X_train, y_train, random_state):
    rf_ = clone(rf)
    rf_.random_state = random_state
    rf_.fit(X_train, y_train)
    baseline = rf_.oob_score_
    #print("baseline " + str(baseline))
    imp = []
    for col in X_train.columns:
        X = X_train.drop(col, axis=1)
        rf_ = clone(rf)
        rf_.random_state = random_state
        rf_.fit(X, y_train)
        o = rf_.oob_score_
        imp.append(baseline - o)
    imp = np.array(imp)
    #I = pd.DataFrame(
    #        data={'Feature':X_train.columns,
    #              'Importance':imp})
    #I = I.set_index('Feature')
    #I = I.sort_values('Importance', ascending=True)
    return imp #I

Handling of Pipelines.

In [ ]:
def build_estimator(name, verbose=0):
    regex = r"fs_(.*)_(.*)"

    matches = re.findall(regex, name)
    print(matches)
    
    # feature selection
    if len(matches) > 0:
        fs_name = matches[0][0]
        print('feature selection method %s' % fs_name)
        clf_name = matches[0][1]
        print('classifier %s' % clf_name)
        steps = []
        if not 'forest' in clf_name:
            steps.append(('scaler', make_column_transformer((FunctionTransformer(), get_categorical_features), # default FunctionTransformer implements identity
                                                 remainder=StandardScaler()))
                        )
        steps.append(('fs', configuration['feature_selection'][fs_name]['fs']))
        steps.append(('clf', configuration['classifier'][clf_name]['clf']))
    
    # no feature selection
    else:
        clf_name = name
        steps = []
        if not 'forest' in clf_name:
            steps.append(('scaler', make_column_transformer((FunctionTransformer(), get_categorical_features), # default FunctionTransformer implements identity
                                                 remainder=StandardScaler()))
                        )
        steps.append(('clf', configuration['classifier'][clf_name]['clf']))
    if verbose > 1:
        print('steps %s' % str(steps))
    
    if len(steps) == 1:
        return steps[0][1]
    else:
        return Pipeline(steps)

def get_feature_names_from_pipeline(pipeline, original_feature_names, verbose=0):
    """get_feature_names not available on Pipeline, should handle what happens to feature names depending on 
    Pipeline steps (in order to be able to compute features importance)
    Mainly scaler which is columnTransformer may change the order of columns.
    If there is feature selection, its support_ param will refer to indices from features with order changed by scaler.
    And normally the classifier step should not change anything to the features list or order, but if feat importance is 
    retrieved from clf, then it must also take into account proper order and list of features names (from scaler+fs if 
    any)
    
    Return
    ------
    
    feature_names: dict
        Dictionnary with pipeline steps as keys (limited to 'scaler', 'fs' or 'clf'), and feature names at corresponding
        steps as list values.
    
    """
    feature_names = {}
    current_feature_names = None
    fs_feature_names = []
    if isinstance(pipeline, Pipeline):
        masked_features = None
        if verbose > 1:
            print('DEBUG features %s' % str(features))
            print('DEBUG features from Pipeline %s' % str(parse_feature_names(best_clf.get_feature_names())))
            print('DEBUG pipeline steps %s' % best_clf.named_steps)
         
        for step in ['scaler', 'fs', 'clf']:
            if step in pipeline.named_steps:
                if step == 'scaler':
                    feature_names['scaler'] = parse_feature_names(pipeline.named_steps['scaler'].get_feature_names())
                    current_feature_names = feature_names['scaler']
                elif step == 'fs':
                    fs = pipeline.named_steps['fs']
                    if isinstance(fs, RFECV):
                        support = fs.support_
                    elif isinstance(fs, SelectFromModel) or isinstance(fs, SelectKBest):
                        support = fs.get_support()
                    if current_feature_names is not None:
                        current_feature_names = current_feature_names[support]
                    else:
                        current_feature_names = original_feature_names[support]
                    feature_names['fs'] = current_feature_names
                elif step == 'clf':
                    if current_feature_names is not None:
                        feature_names['clf'] = current_feature_names
                    else:
                        feature_names['clf'] = original_feature_names
    else:
        feature_names['clf'] = original_feature_names
        
    return feature_names

In [ ]:
def get_feature_df(grid_search, features):
    """
    Return the feature names and coefficients from the final classifier of the
    best pipeline found by GridSearchCV. See https://git.io/vPWLI.

    Params
    ------
    grid_search: GridSearchCV object
        A post-fit GridSearchCV object where the estimator is a Pipeline.
    features: list
        initial feature names

    Returns
    -------
    pandas.DataFrame
        Dataframe of feature name and coefficient values
    """
    features = np.array(features)
    pipeline = grid_search.best_estimator_
    for name, transformer in pipeline.steps:
        if name.startswith('select'):
            X_index = np.arange(len(features)).reshape(1, -1)
            indexes = transformer.transform(X_index).tolist()
            features = features[indexes]
    step_name, classifier = pipeline.steps[-1]
    coefficients, = classifier.coef_
    feature_df = pd.DataFrame.from_items([
        ('feature', features),
        ('coefficient', coefficients),
    ])
    return feature_df

## GridSearch

In [ ]:
def score_forest(estimator, X_, y_):
    """
    Custom scoring function for hyperparameter optimization. In this case, we want to print out the oob score.
    estimator was fitted on train data, X and y here are validation data.
    For OOB we don't need validation data - OOB is inherent to tree building.
    Same for feature importances (with default gini impurity measure done by sklearn).
    """
    #all_feature_importances.append(estimator.feature_importances_) # global variable - ugly trick
    score = estimator.oob_score_
    #print "oob_score_:", score
    return score

def svm_fi(estimator, X_, y_):
    # for SVM dropcol feature importances computation, we use validation data (X_, y_) and train data (X, y) are global.
    #X_train_df = pd.DataFrame(X, columns=feats)
    #y_train_df = pd.DataFrame(y)
    #X_valid_df = pd.DataFrame(X_, columns=feats)
    #y_valid_df = pd.DataFrame(y_)
    #dcs = dropcol_importances_sklearn(clf, X_train_df, y_train_df, X_valid_df, y_valid_df)
    #all_dropcol_feature_importances.append(dcs)
    #print(dcs)
    return 0

def run_cross_val_score(clf, X, y, idx=None, n_jobs=-1, verbose=0):
    if verbose > 1:
        print('run_cross_val_score(clf=%s) % str(clf)')
    #print("gridsearch(clf=%s, modelTarget=%s, param_grid=%s, features=%s, upsample=%s)" 
    #      % (type(clf), modelTarget, param_grid, features, upsample))

    scoring = build_scoring(clf, y)
    
    #if not isinstance(clf, JNCC2Wrapper) and not (isinstance(clf, Pipeline) and isinstance(clf.named_steps['clf'], JNCC2Wrapper)):
    cross_val_scores = cross_validate(clf, X, y, scoring=scoring, cv=10, return_train_score=False, 
                                      n_jobs=n_jobs if not isinstance(get_clf_from_estimator(clf), JNCC2Wrapper) else 1, 
                                      verbose=max(0, verbose-1))

    if verbose > 1:
        print('run_cross_val_score: return %s' % str(cross_val_scores))

    return cross_val_scores


In [ ]:
def run_test(clf, param_grid, data, features, prediction_task, cv=10, repeat_dc=300, test_set=0.1, n_splits=1, 
             splits_mode='shuffle', upsample=None, fi=False, bins=None, strategy=None, force_gs=True, n_jobs=-1, verbose=0):
    if verbose > 1:
        print(('run_test(clf=%s, param_grid=%s, features=%s, cv=%d, n_splits=%d, test_set=%f, splits_mode=%s, upsample=%s,'
              +' bins=%d, strategy=%s, n_jobs=%d)') 
              % (clf.__class__.__name__, str(param_grid), str(features), cv, n_splits, test_set, splits_mode, upsample, 
                 bins, strategy, n_jobs))
    X_test = None
    y_test = None
    create_test_set = test_set is not None and test_set > 0
    best_params = None
    grid = None
    is_jncc = isinstance(get_clf_from_estimator(clf), JNCC2Wrapper)
    if verbose > 2:
        print('run_test: is_jncc %s' % str(is_jncc))
    
    all_scores = []
    all_test_scores = []    
    all_supports = []
    all_test_supports = []
    fi_method = []
    fi_fs = pd.DataFrame(np.zeros((1, len(features_df.columns))), columns=features_df.columns)
    
    splits = prepare_train_data(samples=data, features=features, prediction_task=prediction_task, test_set=test_set, 
                                n_splits=n_splits, splits_mode=splits_mode, upsample=upsample, bins=bins, 
                                strategy=strategy, verbose=verbose)

    for idx, (X, y, X_test, y_test) in enumerate(splits):
        if verbose > 1:
            print("*** Train/Test Split #%d / %d" % (idx, len(splits)-1))
        y = y.astype(int)
        support = np.unique(y, return_counts=True)
        support_test = None
        if y_test is not None:
            y_test = y_test.astype(int)
            support_test = np.unique(y_test, return_counts=True)
        all_supports.append(str(support))
        all_test_supports.append(str(support_test))
        
        # first evaluate best params through grid search for this particular dataset/featureset
        # !!! important, see http://www.jmlr.org/papers/volume11/cawley10a/cawley10a.pdf
        # use 'force_gs' so hyper-parameters search is performed independently on each train split before evaluation on test, which would be unbiased protocol

        grid = gridsearch2(clf, 
                          X, y, 
                          prediction_task, 
                          param_grid=param_grid, 
                          features=features,
                          cv=cv,
                          n_jobs=n_jobs if not is_jncc else 1, # avoid multithreading with JNCCWrapper
                          verbose=verbose)
        best_params = grid.best_params_
        single_best_score = grid.best_score_
        if verbose > 1:
            print('  Gridsearch evaluated best score %s' % (single_best_score))
            print('  Gridsearch best params : %s' % str(best_params))



        if create_test_set:

            # compute cv scores (redundant with gridsearch ?)

            best_clf = grid.best_estimator_ #sklearn.base.clone(grid.best_estimator_)
            val_scores = pd.DataFrame(grid.cv_results_, columns=grid.cv_results_.keys())
            all_scores.append(val_scores.loc[grid.best_index_])

            # compute test scores
            
            labels = sorted(support[0])
            if verbose > 2:
                print("LABELS " + str(labels))            

            #best_clf.fit(X, y) # normally has been fitted by gridsearchcv
            y_pred = best_clf.predict(X_test)
            
            if is_jncc:
                # jncc returns a multi-label proposition
                jncc_result = y_pred
                if y_pred.values.ndim == 1:
                    y_pred = jncc_result
                else:
                    y_pred = jncc_result.iloc(axis=1)[0]

            # retrieve feature selection information, if any was performed
            if type(best_clf) == Pipeline and 'fs' in best_clf.named_steps:
                selected_features = get_feature_names_from_pipeline(best_clf, features)['fs']
                if selected_features is not None:
                    if verbose > 1:
                        print('  Selected features %s' % str(selected_features))
                    for ifeat in selected_features:
                        fi_fs[ifeat] += 1                          
            
            
            test_scores_row = {
                'test_balanced_acc': balanced_accuracy_score(y_test, y_pred),
                #'test_balanced_acc_adjusted': balanced_accuracy_score(y_test, y_pred, adjusted=True),
                'test_f1_macro_weighted': f1_score(y_test, y_pred, average='weighted') }
            for idx, label in enumerate(labels):
                test_scores_row['test_f1_macro_class_%d' % label] = f1_score(y_test, y_pred, labels=[label], average='weighted')
            test_scores_row['test_precision_macro_weighted'] =  precision_score(y_test, y_pred, average='weighted')
            for idx, label in enumerate(labels):
                test_scores_row['test_precision_macro_class_%d' % label] = precision_score(y_test, y_pred, labels=[label], average='weighted')            
            test_scores_row['test_recall_macro_weighted'] =  recall_score(y_test, y_pred, average='weighted')
            for idx, label in enumerate(labels):
                test_scores_row['test_recall_macro_class_%d' % label] = recall_score(y_test, y_pred, labels=[label], average='weighted')     
            #test_scores_row['test_f1_micro'] = f1_score(y_test, y_pred, average='micro')
            if is_jncc:
                test_scores_row['test_mrr'] = feu.compute_mrr(y_test, jncc_result, labels)
                test_scores_row['test_determinacy'] = feu.compute_determinacy(jncc_result)
                # TODO: add scores performed by JNCC2 ? (single accuracy etc)
                
            # Compute dummy classifier scores using different strategies, as a baseline
            for dummy_strat in ['uniform', 'stratified', 'most_frequent']:
                dummy = DummyClassifier(strategy=dummy_strat, random_state=RANDOM_STATE)
                dummy.fit(X, y)
                y_pred = dummy.predict(X_test)
                test_scores_row ['rnd_%s_balanced_acc' % dummy_strat] = balanced_accuracy_score(y_test, y_pred)
                test_scores_row['rnd_%s_f1_macro_weighted' % dummy_strat] = f1_score(y_test, y_pred, average='weighted')
                for idx, label in enumerate(labels):
                    test_scores_row['rnd_%s_f1_macro_class_%d' % (dummy_strat, label)] = f1_score(y_test, y_pred, labels=[label], average='weighted')
                test_scores_row['rnd_%s_precision_macro_weighted' % dummy_strat] =  precision_score(y_test, y_pred, average='weighted')
                for idx, label in enumerate(labels):
                    test_scores_row['rnd_%s_precision_macro_class_%d' % (dummy_strat, label)] = precision_score(y_test, y_pred, labels=[label], average='weighted')            
                test_scores_row['rnd_%s_recall_macro_weighted' % dummy_strat] =  recall_score(y_test, y_pred, average='weighted')
                for idx, label in enumerate(labels):
                    test_scores_row['rnd_%s_recall_macro_class_%d' % (dummy_strat, label)] = recall_score(y_test, y_pred, labels=[label], average='weighted')                  
                test_scores_row['rnd_%s_f1_micro' % dummy_strat] = f1_score(y_test, y_pred, average='micro')
                    
            test_scores_df = pd.DataFrame([test_scores_row], index=['score'])
            all_test_scores.append(test_scores_df)


    if len(all_scores) == 1:
        all_scores = all_scores[0]
    elif len(all_scores) > 0:
        all_scores = pd.concat(all_scores, axis=1).T
    else:
        all_scores = pd.DataFrame()
    if len(all_test_scores) == 1:
        all_test_scores = all_test_scores[0]
    elif len(all_test_scores) > 0:
        all_test_scores = pd.concat(all_test_scores)
    else:
        all_test_scores = pd.DataFrame()
        
    result = {
        'all_scores': all_scores,
        'all_test_scores': all_test_scores,
        'dropcol_fi': all_dropcol_feature_importances,
        'fi': all_feature_importances,
        'fi_fs': fi_fs,
        'support': support,
        'support_test': support_test
    }
    
    if verbose > 2:
        print('run_gridsearch: all_scores =')
        display(HTML(pd.DataFrame(all_scores).to_html()))
        print('run_gridsearch: all_test_scores =')
        display(HTML(pd.DataFrame(all_test_scores).to_html()))
        
    return grid, result

## Test plan utilities

### Gridsearch hyper-parameters ranges

In [ ]:
# random forests
param_grid_rf =  {
    'n_estimators' : [300],
    'max_features': ['sqrt'],
    'max_depth': [None],
    'min_samples_leaf': [1],
    'criterion': ['gini', 'entropy']
}
# for quickly testing the loops
param_grid_rf_tests = {
    'n_estimators' : [3],
    'max_features': ['sqrt'],
    'max_depth': [None],
    'min_samples_leaf': [1],
    'criterion': ['gini', 'entropy']
}
# SVM
param_grid_svm = {
    'clf__kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
    'clf__C': np.logspace(-2, 1, 4),
    'clf__gamma': np.logspace(-3, 1, 5)
}
# Gaussian Naive Bayes
param_grid_gnb = {
    'clf__priors': [None], 
    'clf__var_smoothing': [1e-09]
}

param_grid_jncc2 = {}


### Global configuration

In [ ]:
# TOTO
from sklearn.ensemble import RandomForestRegressor

def get_categorical_features(X):
    if 'Expert' in X.columns:
        return ['Expert']
    else:
        return []
    
ct = make_column_transformer(
    (FunctionTransformer(), get_categorical_features), # default FunctionTransformer implements identity
    remainder=StandardScaler()
) #ERROR #TODO columns in output are in different order than input !!!!!!!!!!!!

configuration = {
    'subject': {
        'doctor': {
            'name': 'Doctor',
            'modes': {
                'verbal': {
                    'dataset': subject_np_df,
                    'name': 'Verbal'
                },
                'nonverbal': {
                    'dataset': subject_np_df,
                    'name': 'Non-Verbal'
                },
                'multimodal': {
                    'dataset': subject_np_df,
                    'name': 'Multimodal'
                },
                'nonverbal-a': {
                    'dataset': subject_np_df,
                    'name': 'Non-Verbal-A'
                },
                'multimodal-a': {
                    'dataset': subject_np_df,
                    'name': 'Multimodal-A'
                },
            },
            'phases': {
                'nophase': {
                    'dataset': subject_np_df,
                    'name': 'No Phase'
                },
                '157015': {
                    'dataset': subject_p_df,
                    'name': '157015'
                },
                'nophase-a': {
                    'dataset': subject_np_df,
                    'name': 'No Phase-A'
                },
                '157015-a': {
                    'dataset': subject_p_df,
                    'name': '157015-A'
                },
                'nophase-full': {
                    'dataset': subject_np_df,
                    'name': 'No Phase-FULL'
                },
                '157015-full': {
                    'dataset': subject_p_df,
                    'name': '157015-FULL'
                }    
            }
        },
        'agent': {
            'name': 'Agent',
            'modes': {
                'verbal': {
                    'dataset': agent_np_df,
                    'name': 'Verbal'
                },
                'nonverbal': {
                    'dataset': agent_np_df,
                    'name': 'Non-Verbal'
                },
                'multimodal': {
                    'dataset': agent_np_df,
                    'name': 'Multimodal'
                },
                'nonverbal-a': {
                    'dataset': agent_np_df,
                    'name': 'Non-Verbal-A'
                },
                'multimodal-a': {
                    'dataset': agent_np_df,
                    'name': 'Multimodal-A'
                },
            },
            'phases': {
                'nophase': {
                    'dataset': agent_np_df,
                    'name': 'No Phase'
                },
                '157015': {
                    'dataset': agent_p_df,
                    'name': '157015'
                },
                'nophase-a': {
                    'dataset': agent_np_df,
                    'name': 'No Phase-A'
                },
                '157015-a': {
                    'dataset': agent_p_df,
                    'name': '157015-A'
                },
                'nophase-full': {
                    'dataset': agent_np_df,
                    'name': 'No Phase-FULL'
                },
                '157015-full': {
                    'dataset': agent_p_df,
                    'name': '157015-FULL'
                }                    
            }            
        },
        'doctor+agent': {
            'name': 'Doctor+Agent',
            'modes': {
                'verbal': {
                    'dataset': all_np_df,
                    'name': 'Verbal'
                },
                'nonverbal': {
                    'dataset': all_np_df,
                    'name': 'Non-Verbal'
                },
                'multimodal': {
                    'dataset': all_np_df,
                    'name': 'Multimodal'
                },
                'nonverbal-a': {
                    'dataset': all_np_df,
                    'name': 'Non-Verbal-A'
                },
                'multimodal-a': {
                    'dataset': all_np_df,
                    'name': 'Multimodal-A'
                },
            },
            'phases': {
                'nophase': {
                    'dataset': all_np_df,
                    'name': 'No Phase'
                },
                '157015': {
                    'dataset': all_p_df,
                    'name': '157015'
                },
                'nophase-a': {
                    'dataset': all_np_df,
                    'name': 'No Phase-A'
                },
                '157015-a': {
                    'dataset': all_p_df,
                    'name': '157015-A'
                },
                'nophase-full': {
                    'dataset': all_np_df,
                    'name': 'No Phase-FULL'
                },
                '157015-full': {
                    'dataset': all_p_df,
                    'name': '157015-FULL'
                }                    
            }            
        }        
    },
    'features': {
        'verbal': 'Verbal',
        'nonverbal': 'Non-Verbal',
        'multimodal': 'Multimodal',
        'multimodal+duration': 'Multimodal+Duration',
        '157015': '157015',
        'nophase': 'No Phase',
        'nonverbal-a': 'Non-Verbal-A',    
        'multimodal-a': 'Multimodal-A',
        '157015-a': '157015-A',
        'nophase-a': 'No Phase-A',
        '157015-full': '157015-FULL',
        'nophase-full': 'No Phase-FULL'
    },
    'classifier': {
        'forest': {
            'clf': RandomForestClassifier(random_state=RANDOM_STATE, oob_score=True),
            'param_grid': param_grid_rf,
            'name': 'RF'
        },
        'forest-test': {
            'clf': RandomForestClassifier(random_state=RANDOM_STATE, oob_score=True),
            'param_grid': param_grid_rf_tests,
            'name': 'RF-test'
        },        
        'r-forest': {
            'clf': RandomForestRegressor(random_state=RANDOM_STATE, oob_score=True),
            'param_grid': param_grid_rf,
            'name': 'RF-REGR'
        },
        'svm-back': {
            'clf': make_pipeline(ct, SVC(random_state=RANDOM_STATE, probability=True)),
            'param_grid': param_grid_svm,
            'name': 'SVM'
        },      
        'gnb-back': {
            'clf': make_pipeline(ct, GaussianNB()),
            'param_grid': param_grid_gnb,
            'name': 'NB-G'
        },
        
        'svm': {
            'clf': SVC(random_state=RANDOM_STATE, probability=True),
            'param_grid': param_grid_svm,
            'name': 'SVM'
        },
        
        'gnb': {
            'clf': GaussianNB(),
            'param_grid': param_grid_gnb,
            'name': 'NB-G'
        },        
        
        'jncc2': {
            'clf': JNCC2Wrapper(),
            'param_grid': param_grid_jncc2,
            'name': 'JNCC2'
            
        },
        'fs_rfe-svm-l2_gnb': {
            'clf': None, #make_pipeline(ct, RFECV(estimator=LinearSVC(penalty='l2'), step=1, cv=10, scoring='f1'), GaussianNB()),  
            'param_grid': param_grid_gnb,
            'name': 'FS_RFE-SVM-L2_NB-G'
        },
        'fs_rfe-svm-l2_svm': {
            'clf': None, #make_pipeline(ct, RFECV(estimator=LinearSVC(penalty='l2'), step=1, cv=10, scoring='f1'), SVC(random_state=RANDOM_STATE, probability=True)),
            'param_grid': param_grid_svm,
            'name': 'FS_RFE-SVM-L2_SVM'
        },        
        'fs_svm-l1_gnb': {
            'clf': None, #make_pipeline(ct, SelectFromModel(estimator=LinearSVC(dual=False, penalty='l1', C=0.1)), GaussianNB()),
            'param_grid': param_grid_gnb,
            'name': 'FS_SVM-L1_NB-G'
        },
        'fs_svm-l1_svm': {
            'clf': None, #make_pipeline(ct, SelectFromModel(estimator=LinearSVC(dual=False, penalty='l1', C=0.1)), SVC(random_state=RANDOM_STATE, probability=True)),
            'param_grid': param_grid_svm,
            'name': 'FS_SVM-L1_NB-G'
        },   
        'fs_kb-chi2_gnb': {
            'clf': None, #make_pipeline(ct, SelectKBest(chi2, k=5), GaussianNB()),
            'param_grid': param_grid_gnb,
            'name': 'FS_KB5-CHI2_NB-G'
        },
        'fs_kb-mi_gnb': {
            'clf': None, #make_pipeline(ct, SelectKBest(mutual_info_classif, k=5), GaussianNB()),
            'param_grid': param_grid_gnb,
            'name': 'FS_KB5-MI_NB-G'
        },           
        'name' : 'Classifier'
    },
    'feature_selection': {
        'rfe-svm-l2': {
            'fs': RFECV(estimator=LinearSVC(penalty='l2'), step=1, cv=10, scoring='f1'),  
            'name': 'RFE-SVM-L2'
        },
        'svm-l1': {
            'fs': SelectFromModel(estimator=LinearSVC(dual=False, penalty='l1', C=0.1)),
            'name': 'SVM-L1'
        },
        'kb-mi': {
            'fs': SelectPercentile(mutual_info_classif, percentile=10),
            'name': 'KB5-MI'
        },           
        'name' : 'Feature Selection'
    },    
    'prediction_task': {
        'presence': 'Presence',
        'copresence': 'Co-Presence'
    }
}


    

### Results utilities

In [ ]:
def get_df_multiindex(test_plan_params, metric_level=True, verbose=0):
    iindex = []
    inames = ['Target', 'Subject']
    
    pred_level = []
    if 'presence' in test_plan_params['prediction_task']:
        pred_level.append('Presence')
    if 'copresence' in test_plan_params['prediction_task']:
        pred_level.append('Co-Presence')
    iindex.append(pred_level)
    
    subj_level = []
    if 'doctor' in test_plan_params['subject']:
        subj_level.append('Doctor')
    if 'agent' in test_plan_params['subject']:
        subj_level.append('Agent')
    if 'doctor+agent' in test_plan_params['subject']:
        subj_level.append('Doctor+Agent')
    iindex.append(subj_level)
        
    if 'modes' in test_plan_params.keys():
        exps = test_plan_params['modes']
        exps_name = 'Mode'
    elif 'phases' in test_plan_params.keys():
        exps = test_plan_params['phases']
        exps_name = 'Phases'

    exp_level = []
    for exp in exps:
        exp_level.append(configuration['features'][exp])
    if len(exp_level) > 0:
        iindex.append(exp_level)
        inames.append(exps_name)

    clf_level = []
    for clf_id in test_plan_params['classifier']:
        clf_level.append(configuration['classifier'][clf_id]['name'])
    if len(clf_level) > 0:
        iindex.append(clf_level)
        inames.append('Classifier')

    if 'upsampling' in test_plan_params.keys():
        up_level = []
        for up_id in test_plan_params['upsampling']:
            up_level.append(str(up_id))
        iindex.append(up_level)
        inames.append('Oversampling')
        
    if 'nbclasses' in test_plan_params.keys():
        cl_level = []
        for cl_id in test_plan_params['nbclasses']:
            cl_level.append(cl_id)
        iindex.append(cl_level)
        inames.append('Nb Classes')
        
    if 'clstrategy' in test_plan_params.keys():
        cls_level = []
        for cls_id in test_plan_params['clstrategy']:
            cls_level.append(str(cls_id))
        iindex.append(cls_level)
        inames.append('Classes discretization strategy')

    if metric_level:
        iindex.append(['score', 'err'])
        inames.append('Metric')

    scores_df_index = iindex
    if verbose > 1:
        print(scores_df_index)
    scores_df_multiindex = pd.MultiIndex.from_product(scores_df_index, 
                                                      names = inames)
    return scores_df_multiindex

def get_results_tuple(test, upsample_strategy_default):
    subject = test['subject']
    subj_conf = configuration['subject'][subject]
    prediction_task = test['prediction_task']
    classifier = test['classifier']
    
    if 'modes' in test.keys():
        fset = test['modes']  
    elif 'phases' in test.keys():
        fset = test['phases']     

    if 'upsampling' in test:
        upsample = test['upsampling']
    else:
        upsample = upsample_strategy_default

    result_tuple = ()
    result_tuple = result_tuple + ('Presence',) if prediction_task == 'presence' else ('Co-Presence',)
    result_tuple = result_tuple + (subj_conf['name'],)
    result_tuple = result_tuple + (configuration['features'][fset],)
    result_tuple = result_tuple + (configuration['classifier'][classifier]['name'],)
    
    if 'upsampling' in test:
        result_tuple = result_tuple + (str(upsample),)
        
    if 'nbclasses' in test:
        result_tuple = result_tuple + (str(test['nbclasses']),)
    
    if 'clstrategy' in test:
        result_tuple = result_tuple + (str(test['clstrategy']),)
        
    return result_tuple



### Test plan runner

In [ ]:
# number of bins for presence / co-presence discretization
n_bins_default = 5
# strategy for presence / co-presence discretization
bin_strategy_default = 'kmeans'
# default upsampling
upsample_strategy_default = 'imb_smotenc'
# whether to apply 'original' upsampling method
#upsample = True
verbose=1
test_set = True
fi = False

all_feature_importances = []    
all_dropcol_feature_importances = []

def run_test_plan(test_plan_params, group, exp, iter, 
                  test_set=0.1, n_splits=1, splits_mode='shuffle', cv=10, repeat_cv=10, 
                  repeat_fi=300, upsample_strategy=upsample_strategy_default, 
                  n_bins=n_bins_default, bin_strategy=bin_strategy_default, 
                  n_jobs=-1, verbose=verbose):
    """
    """
    tp_start = time.time()
    
    ### Create dataframe for results, context, feature importance (from feature selection)
    tests_df_file = os.path.join(config.OUT_PATH, 'tests_executions.xlsx')
    if os.path.isfile(tests_df_file):
        tests_df = pd.read_excel(tests_df_file)
    else:
        tests_df = pd.DataFrame()
    tests_df_cols = ['Test plan', 'Group', 'Experiment', 'Exp. Iteration',  'Test plan size', 'Test set ratio', 
                     'Internal CV', 'External CV', 'External CV mode', 'Upsampling', 'Classes', 'Classes binning', 
                     'Date Start', 'Date End', 'Duration', 'Nb executed tests']
    tests_df_row = pd.Series({
        'Test plan': str(test_plan_params),
        'Group': group,
        'Experiment': exp,
        'Exp. Iteration': iter,
        'Test plan size': len(ParameterGrid(test_plan_params)),
        'Test set ratio': test_set,
        'Internal CV':cv,
        'External CV': n_splits if splits_mode=='shuffle' else n_splits*(int(1/test_set)),
        'External CV mode' : splits_mode,
        'Upsampling': upsample_strategy,
        'Classes': n_bins,
        'Classes binning': bin_strategy,
        'Date Start': datetime.now().strftime("%d/%m/%Y %H:%M:%S"),
        'Date End': None,
        'Duration': None,
        'Nb executed tests': 0
    })
    tests_df = tests_df.append(tests_df_row, ignore_index=True)
    tests_df.to_excel(tests_df_file, index=False, columns=tests_df_cols)
    
    dh = DataHandler(group, exp, iter)
    scores_df_multiindex = get_df_multiindex(test_plan_params)

    scores_df = dh.load_obj('results', 'scores')
    if scores_df is None:
        scores_df = pd.DataFrame(index=scores_df_multiindex)
        dh.save_obj(scores_df, 'results', 'scores')
    context_df = dh.load_obj('.', 'context')
    if context_df is None:
        ctx_df_multiindex = get_df_multiindex(test_plan_params, metric_level=False)
        if verbose > 2:
            print('CTX multiindex %s' %  str(ctx_df_multiindex))
        context_df = pd.DataFrame(index=ctx_df_multiindex)
        #context_df['Best Params'] = ''
        #context_df['Best Params'] = context_df['Best Params'].apply(str)
        dh.save_obj(context_df, '.', 'context')
    fi_fs_df = dh.load_obj('results', 'fi_fs')
    if fi_fs_df is None:
        fi_fs_df_multiindex = get_df_multiindex(test_plan_params, metric_level=False)
        fi_fs_df = pd.DataFrame(index=fi_fs_df_multiindex, columns=features_df.columns)
        dh.save_obj(fi_fs_df, '.', 'fi_fs')
    
    ### Execute test plan and fill results dataframes
    
    test_plan = ParameterGrid(test_plan_params)
    if verbose > 0:
        print("%d tests to be performed" % len(test_plan))
        display(HTML(pd.DataFrame(list(test_plan)).to_html()))
        
        
    for idx, test in enumerate(test_plan):
        if verbose > 0:
            print("########## TEST %d / %d : %s" % (idx, len(test_plan), test))

        #try:
        #print("before")
        # make it global because we use them to compute feature importance for forests
        all_feature_importances = []    
        all_dropcol_feature_importances = []

        subject = test['subject']
        prediction_task = test['prediction_task']
        classifier = test['classifier']

        if 'upsampling' in test:
            upsample = test['upsampling']
        else:
            upsample = upsample_strategy
            
        if 'nbclasses' in test:
            n_bins = test['nbclasses']
        if 'clstrategy' in test:
            bin_strategy = test['clstrategy']

        #clf = configuration['classifier'][classifier]['clf']
        clf = build_estimator(classifier)
        pgrid = configuration['classifier'][classifier]['param_grid']
        subj_conf = configuration['subject'][subject]
        if 'modes' in test.keys():
            fset = test['modes']
            df = subj_conf['modes'][fset]['dataset']
            feats = get_features_set(subj_conf['name'], subj_conf['modes'][fset]['name'])      
        elif 'phases' in test.keys():
            fset = test['phases']
            df = subj_conf['phases'][fset]['dataset']
            feats = get_features_set(subj_conf['name'], subj_conf['phases'][fset]['name'])

        if upsample not in (None, 'imb_smotenc', 'imb_random') and 'Expert' in feats:
            print ('ERROR: only SMOTE-NC and RANDOM strategies are supported with categorical features, skipping test')
            continue
            
        # regression task
        if classifier.startswith('r-'):
            n_bins = 0
            upsample = None
        # JNCC2
        if isinstance(clf, JNCC2Wrapper):
            # pass the DataHandler
            clf = JNCC2Wrapper(dh)
        elif isinstance(clf, Pipeline):
            if 'clf' in clf.named_steps and isinstance(clf.named_steps['clf'], JNCC2Wrapper):
                # pass the DataHandler to the pipelined JNCC2
                clf.set_params(clf__dataHandler=dh) 
                if verbose > 1:
                    print('DEBUG dh root_path %s' % clf.named_steps['clf'].dataHandler.root_path)
            if verbose > 1:
                print('DEBUG pipeline named steps %s' % str(clf.named_steps))  
            

        #print("after")

        start = time.time()

        #X, y = prepare_train_data(samples=df, features=feats, target_model=target, upsample=upsample, test_set=True) 

        grid, result = \
            run_test(
                    clf, 
                    pgrid, 
                    df, feats, 
                    prediction_task, 
                    cv=cv, 
                    upsample=upsample, 
                    test_set=test_set,
                    n_splits=n_splits,
                    splits_mode=splits_mode,
                    fi=fi,
                    bins=n_bins, strategy=bin_strategy,
                    n_jobs=n_jobs, verbose=verbose)
        scores, test_scores, dropcol_dis, fis = result['all_scores'], result['all_test_scores'], result['dropcol_fi'], result['fi']
        support, support_test, fi_fs = result['support'], result['support_test'], result['fi_fs']
        #print(scores.mean(axis='columns'))
        #print('TEST SCORES (RAW):')
        #print(test_scores)
        if verbose > 1:
            display(HTML(scores.to_html()))

        elapsed = time.time() - start
        if verbose > 0:
            print("   Elapsed: %s seconds" % str(elapsed))

        isDoc = 'doctor' in subject
        isAgent = 'agent' in subject

        results_tuple = get_results_tuple(test, upsample_strategy_default=upsample_strategy)
        


        # compute err

        cols_to_remove = [col for col in list(scores.columns) if 'time' in col or 'split' in col or 'rank' in col or 'param' in col]
        if verbose > 2:
            print('scores columns: %s' % str(scores.columns))
            print('scores columns to remove    : %s' % str(cols_to_remove))
        all_scores_ = scores.drop(cols_to_remove, axis=1).T
        if verbose > 2:
            print('scores new columns: %s' % str(all_scores_.columns))
            display(HTML(all_scores_.to_html()))
        #!!!TODO: pour gridsearch, calculer le mean (sur 10xCVs = OK) et la std 
        # (racine de var globale = var intra+var inter) = PAS OK
        mean_cols = [col for col in list(all_scores_.index) if 'mean' in col]
        std_cols = [col for col in list(all_scores_.index) if 'std' in col]
        all_scores_means = all_scores_.loc[mean_cols].mean(axis=1).rename(index={name:name.replace('mean_', '') 
                                                                                 for name in mean_cols})
        inter_var = all_scores_.loc[mean_cols].var(axis=1)
        inter_var = inter_var.rename(index={name:name.replace('mean_', '') for name in mean_cols})
        intra_var = np.square(all_scores_.loc[std_cols]).mean(axis=1)
        intra_var  = intra_var.rename(index={name:name.replace('std_', '') for name in std_cols})
        global_var = inter_var + intra_var
        all_scores_cit = global_var / np.sqrt(3*10) * 1.96        
        
        final = pd.DataFrame(index=all_scores_cit.index)

        if verbose > 2:
            print('mean cols %s, std cols %s' % (mean_cols, std_cols))
        final['score'] = all_scores_means
        #final['std'] = 
        final['err'] = all_scores_cit     
    
        final.index = [row.replace('test_', 'val_') for row in final.index]
        if verbose>2:
            print('score and err for gs results:')
            display(HTML(final.to_html()))        
        
        
        if test_scores is not None and not test_scores.empty:
            test_scores_df = pd.DataFrame()
            #!!! pour les scores de TEST ce calcul est OK par contre
            test_scores_df['score'] = test_scores.mean(axis=0)
            test_scores_df['err'] = test_scores.std(axis=0) # pour être cohérent il faudrait calculer l'intervalle de confiance !
            #print("TEST scores:")
            #display(HTML(test_scores_df.to_html()))

            final_ = pd.concat([final, test_scores_df], axis=0, join='outer').T
        else:
            final_ = final.T
        #display(HTML(final_.to_html()))
        final = pd.DataFrame(index=scores_df_multiindex, columns=final_.columns)
        #display(HTML(final.to_html()))
        final.loc[results_tuple + ('score',)] = final_.loc['score']
        final.loc[results_tuple + ('err',)] = final_.loc['err']

        final.dropna(axis='rows', how='all', inplace=True)
        #display(HTML(final.to_html()))
        if verbose > 1:
            print("      updating scores for %s" % str(results_tuple))

        # insert missing columns in results df if needed
        s = set(final.columns) - set(scores_df.columns)
        #print("DEBUG inserting new columns %s" % str(s))
        new = list(sorted(s))
        for newcol in new:
            scores_df[newcol] = [np.nan]*len(scores_df)

        #display(HTML(scores_df.dropna(axis='rows', how='all', inplace=False).to_html()))
        scores_df.loc[results_tuple + ('score',)] = pd.Series(final.loc[results_tuple + ('score',)].to_dict())
        scores_df.loc[results_tuple + ('err',)] = pd.Series(final.loc[results_tuple + ('err',)].to_dict())
        #display(HTML(scores_df.dropna(axis='rows', how='all', inplace=False).to_html()))  
        dh.save_obj(scores_df, 'results', 'scores')

        dumpPath = os.path.join(dh.root_path, 'results', 'scores.xlsx')
        scores_df.to_excel(dumpPath, index=True)
        
        if grid is not None:
            best_params = grid.best_params_
        else:
            best_params = list(ParameterGrid(pgrid))
            
        
        
        new_row = pd.Series({
            'Duration': elapsed,                
            'Support': str(support), 
            'Support (test)': str(support_test), 
            'Oversampling': str(upsample),             
            'Y nb classes': str(n_bins),
            'Y strategy': str(bin_strategy),
            'Test set ratio': str(test_set),
            'Grid search params space': str(pgrid),
            'Best Params': str(best_params),            
            'Features': str(feats)
        })
        ordered_cols = ['Duration', 'Support', 'Support (test)', 'Oversampling', 
                        'Y nb classes', 'Y strategy', 'Test set ratio', 'Grid search params space', 'Best Params',
                        'Features']
        # insert missing columns in context df if needed
        s = set(new_row.index) - set(context_df.columns)
        new = list(s)
        for newcol in new:
            context_df[newcol] = [np.nan]*len(context_df)
        context_df.loc[results_tuple] = new_row
        dumpPath = os.path.join(dh.root_path, '.',  'context.csv')
        context_df.to_csv(dumpPath, index=True)

        if np.mean(fi_fs.values, axis=None) > 0:
            fi_fs_df.loc[results_tuple] = fi_fs.values
            dumpPath = os.path.join(dh.root_path, 'results', 'fi_fs.csv')
            fi_fs_df[ordered_cols].to_csv(dumpPath, index=True)

        #except Exception as e:
        #    print("ERROR: exception occurred, skiping test")
        #    print(e)
        tests_df.iloc[len(tests_df)-1, tests_df.columns.get_loc('Nb executed tests')] = idx+1
        tests_df.iloc[len(tests_df)-1, tests_df.columns.get_loc('Date End')] = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        tests_df.to_excel(tests_df_file, index=False, columns=tests_df_cols)
    
    tp_end = time.time()
    tests_df.iloc[len(tests_df)-1, tests_df.columns.get_loc('Duration')] = tp_end - tp_start
    tests_df.to_excel(tests_df_file, index=False, columns=tests_df_cols)
    
    return dh, grid


### Plots

In [ ]:
def plot_importance(title, importances, importances_err, feature_names, sort=True):
    # Sort feature importances in descending order
    if sort:
        indices = np.argsort(importances)[::-1]
    else:
        indices = np.arange(len(feature_names))

    # Rearrange feature names so they match the sorted feature importances
    names = [feature_names[i] for i in indices]
    if importances_err is not None:
        errs = [importances_err[i] for i in indices]

    # Create plot
    plt.figure()

    # Create plot title
    plt.title("Feature Importance - " + title)

    # Add bars
    if importances_err is not None:
        plt.bar(range(len(feature_names)), importances[indices], yerr=errs, xerr=0.1)
    else:
        plt.bar(range(len(feature_names)), importances[indices])

    # Add feature names as x-axis labels
    plt.xticks(range(len(feature_names)), names, rotation=90)

    dh.save_fig(title)
    #plt.savefig(os.path.join(dh.root_path, 'figures', title+'.png'))
    plt.show()


In [ ]:
# Some utilities for plots


def add_line(ax, xpos, ypos):
    line = plt.Line2D([xpos, xpos], [ypos + .1, ypos],
                      transform=ax.transAxes, color='gray')
    line.set_clip_on(False)
    ax.add_line(line)

def label_len(my_index,level):
    labels = my_index.get_level_values(level)
    return [(k, sum(1 for i in g)) for k,g in groupby(labels)]

def label_group_bar_table_(ax, df, nlevels=-1):
    ypos = -.1
    scale = 1./df.index.size
    if nlevels == -1:
        nlevels = df.index.nlevels
    
    for level in range(nlevels)[::-1]:
        pos = 0
        for label, rpos in label_len(df.index,level):
            lxpos = (pos + .5 * rpos)*scale
            ax.text(lxpos, ypos, label, ha='center', transform=ax.transAxes)
            add_line(ax, pos*scale, ypos)
            pos += rpos
        add_line(ax, pos*scale , ypos)
        ypos -= .1
        
def label_group_bar_table(ax, df, nlevels=-1, fixed_levels = [], verbose=0):
    ypos = -.1
    scale = 1./df.index.size
    if nlevels == -1:
        nlevels = df.index.nlevels
        
    index_spec = df.index
    levels_to_drop = []
    for idx, level in enumerate(df.index.levels):
        if verbose > 0:
            print('debug cur level %d %s / cur unique value %s' % (idx, level, np.unique(df.index.get_level_values(idx).values)))
        if len(np.unique(df.index.get_level_values(idx).values)) == 1 and idx not in fixed_levels:
            if verbose > 0:
                print('debug drop level ' + str(idx))
            levels_to_drop.append(idx)
            #index_spec = df.index.droplevel(idx)
            nlevels -= 1
    df.index = df.index.droplevel(levels_to_drop) 
    if verbose > 0:
        print('debug df.index ' + str(df.index))
    levels_lens = [ len(np.unique(df.index.get_level_values(idx).values)) for idx,_ in enumerate(df.index.levels)]
    if verbose > 0:
        print('debug levels lens %s ' % levels_lens)
    sorted_levels = np.arange(len(df.index.levels))[np.argsort(levels_lens)]
    if verbose > 0:
        print('debug sorted levels by len %s' % sorted_levels)
    df.index = df.index.reorder_levels(sorted_levels)
    for level in range(nlevels)[::-1]:
        if verbose > 0:
            print('debug level '+str(level))
        pos = 0
        for label, rpos in label_len(df.index,level):
            if verbose > 0:
                print('debug label %s rpos %s' % (str(label), str(rpos)))
            lxpos = (pos + .5 * rpos)*scale
            ax.text(lxpos, ypos, label, ha='center', transform=ax.transAxes)
            add_line(ax, pos*scale, ypos)
            pos += rpos
        add_line(ax, pos*scale , ypos)
        ypos -= .1        

In [ ]:
def plot_leveled_bars(df, df_err, levels, title):
    ax = df \
        .plot(kind='bar', figsize=(10,5), yerr=df_err,
              stacked=False,
              title=title,
              capsize=2)
    ax.set_xticklabels('')
    ax.set_xlabel('')
    ax.grid(axis='y')
    label_group_bar_table(ax, df, levels)
    return ax

def get_selector_tuple(df, prediction_task_level, classifier_level, prediction_task, classifier):
    selector = []
    multi_levels_nb = 0
    for idx, name in enumerate(df.index.names[0:-1]):
        if idx == prediction_task_level:
            selector.append(prediction_task)
        elif idx == classifier_level:
            selector.append(classifier)
        else:
            selector.append(slice(None))
            multi_levels_nb += 1
    return tuple(selector), multi_levels_nb

def plot_results(dh, df, title_prefix, prediction_task_level, classifier_level, repeats=100, verbose=0):
    
    columns_spec = {
        'train': [col for col in df.columns if 'train_' in col and 'class' not in col and 'fi' not in col],
        'train (classes)': [col for col in df.columns if 'train_' in col and 'class' in col and 'fi' not in col], 
        'val': [col for col in df.columns if 'val_' in col and 'class' not in col and 'fi' not in col],
        'val (classes)': [col for col in df.columns if 'val_' in col and 'class' in col and 'fi' not in col],
        'test': [col for col in df.columns if 'test_' in col and 'class' not in col and 'fi' not in col],
        'test (classes)': [col for col in df.columns if 'test_' in col and 'class' in col and 'fi' not in col]
    }
    
    width = 0.095
    w = 0
    N = len(df)
    ind = np.arange(N) + .15
    cmap = plt.cm.tab10.colors
    
    
    prediction_tasks = ['all']
    classifiers = ['all']
    if prediction_task_level is not None:
        prediction_tasks = df.index.levels[prediction_task_level]
    if classifier_level is not None:
        classifiers = df.index.levels[classifier_level]
        
    
    for cset, columns in columns_spec.items():
        
        for prediction_task in prediction_tasks:

            for classifier in classifiers:
                      
                try:
                    columns = sorted(columns)
                    width = 0.7 / len(columns)
                    w = 0

                    s, multi_levels_nb = get_selector_tuple(df, prediction_task_level, classifier_level, prediction_task, classifier)
                    if verbose > 0:
                        print('debug s %s multi %s' % (str(s), str(multi_levels_nb)))
                    multi_levels_nb=len(df.index.levels)

                    if 'test' in cset:
                        # choose random classifier with best f1 score for display
                        f1_rnd_scores = []
                        f1_rnd_names  = []
                        for i in ['most_frequent', 'uniform', 'stratified']:
                            if 'rnd_%s_f1_macro_weighted' % i in df.columns:
                                f1_rnd_scores.append(df.loc(axis=0)[s+('score',)]['rnd_%s_f1_macro_weighted' % i].mean())
                                f1_rnd_names.append(i)
                        best_rnd_strat = f1_rnd_names[np.argmax(f1_rnd_scores)]
                        #print('(best random classifier strategy: %s)' % best_rnd_strat)               

                    #print('multi levels nb ' + str(multi_levels_nb))
                    if 'classes' in cset:
                        figsize=(30,15)
                    else:
                        figsize=(20,10)
                    fig, ax = plt.subplots(figsize=figsize)
                    score_df = df.loc(axis=0)[s + ('score',)]
                    if verbose > 0:
                        display(HTML(score_df.to_html()))
                    err_df = df.loc(axis=0)[s + ('err',)]
                    #display(HTML(err_df.to_html()))
                    N = len(score_df)
                    ind = np.arange(N) - width * len(columns) / 2.0 #+ .15
                    ymin = 1
                    first_rnd = True
                    for i,n in enumerate(columns):
                        rect1 = ax.bar(ind+w, score_df[n], float(width), color=cmap[i%len(cmap)], label=n, yerr=err_df[n] / np.sqrt(repeats), capsize=3)
                        #print('STD %s ERR %s' % (str(err_df[n]), str(err_df[n] / np.sqrt(repeats))))
                        if np.amin(score_df[n]) < ymin:
                            ymin = np.amin(score_df[n])
                        if 'test' in cset:
                            rnd_col = n.replace('test_', 'rnd_%s_' % best_rnd_strat)
                            if rnd_col in df.columns:
                                if first_rnd:
                                    label_rnd = 'random classifier (strategy: %s)' % best_rnd_strat
                                    first_rnd = False
                                else:
                                    label_rnd = None
                                rect2 = ax.bar(ind+w, score_df[rnd_col], float(width), color='lightgray', label=label_rnd,alpha=0.3, yerr=err_df[rnd_col] / np.sqrt(repeats), capsize=3, ecolor='darkgray')
                                if np.amin(score_df[rnd_col]) < ymin:
                                    ymin = np.amin(score_df[rnd_col])
                        w += width
                    ax.set_xticklabels('')
                    ax.set_xlabel('')
                    ax.grid(axis='y')
                    label_group_bar_table(ax, score_df, multi_levels_nb, [prediction_task_level, classifier_level])
                    ax.autoscale_view()
                    plt.xlim(-0.5, N-0.5)
                    #print(ymin-0.1)
                    plt.ylim(bottom=np.max([0, ymin-0.1]))
                    plt.title('%s for %s prediction (%s) (%s)' % (title_prefix, prediction_task, classifier, cset))
                    plt.legend()
                    dh.save_fig(plt, '%s_%s_%s_%s.png' % (title_prefix.replace(' ', '-'), prediction_task, classifier, cset.replace(' ', '-')))
                    plt.show()
                except Exception as e:
                    print('<<no data for %s %s>>' % (prediction_task, classifier))
                    print(e)
                    
                    
                    
# test      
try:
    dh = DataHandler('classifiers-comparison', 'bins-2-uniform_repeats-100', 1)
    scores_df = dh.load_obj('results', 'scores')
    scores_df = scores_df.dropna(axis='rows', how='all')
    display(HTML(scores_df.to_html()))
    plot_results(dh, scores_df, 'Classifiers', 0, 3)
except Exception as e:
    print('No data to display for %s' % dh.root_path)
    print(e)


# Experiments

## Random Forests - n_estimators evaluation
First for Random Forests Classifier, we study when out of bag score stabilizes for a range of n_estimators values.
We will use found n_estimator value with good balance between score and reliability, and performances.

Note: we hypothetize that we can find this value on a specific feature set (subject doctor, no phase) and use it for other sets of features, given overall domain and task remain similar.

In [ ]:
def oob_score(estimator, X, y):
    """
    Custom scoring function for hyperparameter optimization. In this case, we want to print out the oob score.
    estimator was fitted on train data, X and y here are validation data.
    For OOB we don't need validation data - OOB is inherent to tree building.
    Same for feature importances (with default gini impurity measure done by sklearn).
    We add to this the 
    """
    score = estimator.oob_score_
    return score


splits = prepare_train_data(all_p_df, get_features_set('Doctor', '157015'), prediction_task='presence', 
                                          upsample='imb_smotenc', bins=2, strategy='uniform', test_set=0.1, n_splits=1)
(X, y, X_test, y_test) = splits[0]
#n_estimators_range = np.arange(1,100, 10)#
n_estimators_range = np.unique(np.logspace(0, 3, 100, dtype=int)) #TODO launch with exp+5 and more values
scores = []
errs = []
test_acc = []
test_acc_adjusted = []
test_f1 = []
test_precision = []
test_recall = []
scoring = {
    'oob' : oob_score,
    'f1_macro' : 'f1_macro',
    'precision_macro' : 'precision_macro',
    'recall_macro' : 'recall_macro'         
}

from tqdm import tqdm, tqdm_notebook

pbar = tqdm_notebook(n_estimators_range, desc='n_estimators')
for i in pbar:
    pbar.set_description('n_estimators = %d / %s' % (i, n_estimators_range[-1]))
    clf = RandomForestClassifier(n_estimators=int(i), oob_score=True)
    result = cross_validate(clf, X, y, scoring=scoring, return_train_score=False, cv=5, verbose=0)
    scores.append(pd.DataFrame(result).mean().values)
    errs.append(pd.DataFrame(result).std().values)
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    test_acc.append(balanced_accuracy_score(y_test, y_pred))
    test_acc_adjusted.append(balanced_accuracy_score(y_test, y_pred, adjusted=True))
    test_f1.append(f1_score(y_test, y_pred, average='weighted'))
    test_precision.append(precision_score(y_test, y_pred, average='weighted'))
    test_recall.append(precision_score(y_test, y_pred, average='weighted'))

In [ ]:
df = pd.DataFrame(scores, columns=pd.DataFrame(result).columns)
dferr = pd.DataFrame(errs, columns=pd.DataFrame(result).columns)
df = df.drop(['fit_time', 'score_time'], axis=1)
dferr = dferr.drop(['fit_time', 'score_time'], axis=1)
plt.figure(figsize=(20,10))
plt.hlines(y=df['test_oob'][400:].mean(), xmin=0.0, xmax=1000, color='black')
for key in df.columns:
    plt.plot(n_estimators_range, df[key], label=key.replace('test_', 'val_'))
    plt.fill_between(n_estimators_range, 
                     df[key]+(dferr[key]/np.sqrt(len(dferr))), 
                     df[key]-(dferr[key]/np.sqrt(len(dferr))),
                     alpha=0.1)
plt.plot(n_estimators_range, test_acc, '--', label='Test accuracy')
plt.plot(n_estimators_range, test_acc_adjusted, '--', label='Test accuracy (adjusted)')
plt.plot(n_estimators_range, test_f1, '--', label='Test f1 (weighted)')
plt.plot(n_estimators_range, test_precision, '--', label='Test precision (weighted)')
plt.plot(n_estimators_range, test_recall, '--', label='Test recall (weighted)')
plt.xticks(n_estimators_range[::4], n_estimators_range[::4])
plt.legend()
plt.xlabel('n_estimators')
plt.title('Random Forests results depending on number of trees')
dh = DataHandler('common', 'nb_trees_2_uniform')
dh.save_fig(plt, 'RF_nb_trees_2bins-uniform-smotenc_detailed')

## Classes discretization experiments

In [ ]:
test_plan_params = {
    'classifier': ['gnb'],
    'subject': ['doctor'],
    'prediction_task': ['presence', 'copresence'],
    'phases': ['nophase-full'],
    'upsampling': [None],
    'clstrategy': [None, 'uniform', 'kmeans', 'quantile'],    
    'nbclasses': [2, 3, 4, 5]
}

In [ ]:
RANDOM_STATE=42
dh = run_test_plan(test_plan_params, '_discretization', 'NB-G_pres-copres_doctor_noup', 1, 
                   n_bins=None, bin_strategy=None, n_splits=100, test_set=0.1, n_jobs=-1, verbose=1)

In [ ]:
dh = DataHandler('_discretization', 'NB-G_pres-copres_doctor_noup', 1)
df = dh.load_obj('results', 'scores')
df = df.dropna(axis='rows', how='all')

plot_results(dh, df, 'Discretization', df.index.names.index('Target'), 
             df.index.names.index('Classifier'), verbose=0)

## Comparison of classifiers

We will compare 4 classifiers:

* Random (varying strategies)
* Naïve Bayes
* Naïve Credal
* SVM
* Random Forests

Random classifier will define a bottom-line. We compute random classifiers scores for different strategies (most-frequent, uniform, stratified) and keep the one with best f1 score for comparison.
Naïve Bayes will define our baseline, because it is a very simple and effective classifier that is also very fast.
So we will effectively compare and keep the best among Naïve Credal, SVM and Random Forests.

In [ ]:
test_plan_params = {
    'classifier': ['gnb', 'jncc2', 'svm', 'forest'],#, 'forest'],
    'subject': ['doctor'],
    'prediction_task': ['presence', 'copresence'],
    'phases': ['nophase-full'],
    'upsampling': [None]
}

In [ ]:
RANDOM_STATE=42
dh, grid = run_test_plan(test_plan_params, 'classifiers-comparison', 'bins-2-uniform_repeats-100', 5, 
                   test_set=0.1, n_bins=2, bin_strategy='uniform', n_splits=10, splits_mode='kfold', n_jobs=-1, verbose=1)

In [ ]:
res = grid.cv_results_
res = pd.DataFrame(res)
cols_to_remove = [col for col in res.columns if 'time' in col or 'split' in col or 'rank' in col or 'param' in col]
res = res.drop(cols_to_remove, axis=1).T
display(HTML(res.to_html()))

mean_cols = [col for col in list(res.index) if 'mean' in col]
std_cols = [col for col in list(res.index) if 'std' in col]
print('0 means')
print(res.loc[mean_cols].mean(axis=1).rename(index={name:name.replace('mean_', '') for name in mean_cols}))
print()
print('1 inter var')
inter_var = res.loc[mean_cols].var(axis=1)
inter_var = inter_var.rename(index={name:name.replace('mean_', '') for name in mean_cols})
print(np.square(res.loc[mean_cols].std(axis=1)))
print(inter_var)
print()
print('2 intra var')
intra_var = np.square(res.loc[std_cols]).mean(axis=1)
intra_var  = intra_var.rename(index={name:name.replace('std_', '') for name in std_cols})
print(intra_var)
print()
print('3 global var')
global_var = inter_var + intra_var
print(global_var)

print()
print('4 confidence interval 95%')
ci_95 = global_var / np.sqrt(3*10) * 1.96
print(ci_95)
#print(np.sqrt((res.loc[mean_cols].var(axis=1))
#                        + np.square(res.loc[std_cols]).mean(axis=1)
#               )) / np.sqrt(3*10) * 1.96)

In [ ]:
dh = DataHandler('classifiers-comparison', 'bins-2-uniform_repeats-100', 5)
df = dh.load_obj('results', 'scores')
df = df.dropna(axis='rows', how='all')
 
plot_results(dh, df, 'Classifiers comparison', 0, None)

## JNCC2 tests

In [ ]:
test_plan_params = {
    'classifier': ['jncc2', 'gnb'],
    'subject': ['doctor'],
    'prediction_task': ['presence', 'copresence'],
    'phases': ['nophase-full', '157015-full'],
    'upsampling': [None]
}

In [ ]:
RANDOM_STATE=42

"""logging.basicConfig(filename=config.LOGFILE, level=logging.DEBUG,
                        format='%(levelname)s : %(asctime)s : %(name)s : %(message)s')
logger = logging.getLogger('notebook')
console_handler = logging.StreamHandler()
logger.addHandler(console_handler)"""

dh = run_test_plan(test_plan_params, 'jncc2', 'pres_bins-2-uniform_repeats-100', 2, 
                   n_bins=2, bin_strategy='uniform', n_splits=100, n_jobs=1, verbose=10)

In [ ]:
arr = np.random.randint(2, size=(10,2))
df = pd.DataFrame(arr, columns=['a', 'b'])
print(df['a'].to_numpy()[:, np.newaxis].shape)
jncc_result = np.random.randint(2, size=(10))
print(jncc_result)
determ = np.sum(np.sum(jncc_result != 6666, axis=1) == 1) / float(len(jncc_result))
print(determ)

In [ ]:
dh = DataHandler('jncc2', 'pres_bins-2-uniform_repeats-100', 2)
df = dh.load_obj('results', 'scores')
df = df.dropna(axis='rows', how='all')
 
plot_results(dh, df, 'Naive Bayes and Naive Credal', 0, 3)

## Experiment: entropy versus angular speed as non-verbal features

In [ ]:
test_plan_params = {
    'classifier': ['gnb'],
    'subject': ['doctor'],
    'prediction_task': ['presence', 'co-presence'],
    'modes': ['nonverbal', 'nonverbal-a'],
    'upsampling': [None]
}
test_plan_params = {
    'classifier': ['gnb'],
    'subject': ['doctor'],
    'prediction_task': ['presence'],
    'modes': ['nonverbal-a'],
    'upsampling': [None]
}
dh = run_test_plan(test_plan_params, '__tests', 'metrics_no_gs', 0, 
                   n_bins=2, bin_strategy='uniform', n_splits=100, verbose=2)

In [ ]:
dh = DataHandler('__tests', 'metrics_no_gs', 0)
df = dh.load_obj('results', 'scores')
df = df.dropna(axis='rows', how='all')
 
plot_results(dh, df, 'Test - metrics', 0, 3)

In [ ]:
RANDOM_STATE=42
dh = run_test_plan(test_plan_params, 'entropy_vs_angularSpeed', 'pres_bins-2-uniform_repeats-100', 1, n_bins=2, 
                   bin_strategy='uniform', n_splits=100, verbose=1)

In [ ]:
dh = DataHandler('entropy_vs_angularSpeed', 'pres_bins-2-uniform_repeats-100', 1)
df = dh.load_obj('results', 'scores')
df = df.dropna(axis='rows', how='all')
 
plot_results(dh, df, 'Entropy vs Angular speed features', 0, 3)

## Experiment: upsampling strategy

In [ ]:
test_plan_params = {
    'classifier': ['gnb'],
    'subject': ['doctor', 'agent', 'doctor+agent'],
    'prediction_task': ['presence', 'copresence'],
    'phases': ['nophase-a', '157015-a'],
    'upsampling': [None, 'imb_random', 'imb_smotenc']
}

In [ ]:
RANDOM_STATE=42
dh = run_test_plan(test_plan_params, 'oversampling_method-a', 'pres_bins-2-uniform_repeats-100', 2, 
                   n_bins=2, bin_strategy='uniform', n_splits=100, verbose=1)

In [ ]:
dh = DataHandler('oversampling_method-a', 'pres_bins-2-uniform_repeats-100', 2)
df = dh.load_obj('results', 'scores')
df = df.dropna(axis='rows', how='all')

#scores_df = pd.read_excel(os.path.join(dh.root_path, 'results', 'scores.xlsx'), index_col=[0,1,2,3,4,5])
#display(HTML(scores_df.to_html()))
# .loc(axis=0)[(slice(None,None,None),slice(None,None,None),slice(None,None,None),slice(None,None,None),'None',slice(None,None,None))]
plot_results(dh, df, 'Oversampling method', 0, 3)#
#plot_results(dh, df, 'Oversampling method', 0, None)

In [ ]:
RANDOM_STATE=42
dh_upsampling = run_test_plan(test_plan_params, 'upsampling_strategy', 'bins_3_kmeans_50repeats', 0, n_bins=3, bin_strategy='kmeans', n_splits=100)

## TEST feature selection

In [ ]:
test_plan_params = {
    'classifier': ['gnb', 'fs-svml1+gnb', 'fs-svm+gnb'],
    'subject': ['doctor+agent'],
    'prediction_task': ['presence', 'copresence'],
    'phases': ['nophase-a', '157015-a'],
    'upsampling': [None, 'imb_random', 'imb_smotenc']
}

In [ ]:
RANDOM_STATE=42
dh = run_test_plan(test_plan_params, '__tests', 'feature-selection_gnb', 0, n_bins=2, bin_strategy='uniform', n_splits=100)

In [ ]:
dh = DataHandler('__tests', 'feature-selection_gnb', 0)
scores_df = dh.load_obj('results', 'scores')
scores_df = scores_df.dropna(axis='rows', how='all')


plot_results(dh, scores_df, 'Feature Selection + Gaussian Naive Bayes', 0, None)

## Regression

In [ ]:
test_plan_params_upsampling = {
    'classifier': ['r-forest'],
    'subject': ['doctor+agent'],
    'prediction_task': ['presence'],
    'modes': ['multimodal'],
    'upsampling': [None]
}

RANDOM_STATE=42
dh_upsampling = run_test_plan(test_plan_params_upsampling, 'regression', 'upsampling_strategy', 0, n_bins=0, bin_strategy='kmeans', n_splits=1)

In [ ]:
from itertools import combinations

data = np.arange(1,100)
data_y = np.random.choice(np.arange(1, 4), size=len(data), p=[0.1, 0.4, 0.5])
print(data_y)
print(np.unique(data_y, return_counts=True))

In [ ]:
a = np.where(data_y == 1)[0]
b = np.where(data_y == 2)[0]
c = np.where(data_y == 3)[0]
print('a ' + str(a))
print('b ' + str(b))
print('c ' + str(c))

test_ratio = 0.2
test_size = int(len(data) * test_ratio)
train_size = int(len(data) * (1 - test_ratio))
print
print('train %d train %d' % (train_size, test_size))
def get_exhaustive_splits(arr, test_ratio):
    test_arr_size = int(len(arr) * test_ratio)
    print(test_arr_size)
    test_a = list(combinations(arr, test_arr_size))
    test_a = [list(item) for item in test_a]
    train_a = []
    for comb in test_a:
        train_a.append(list(set(arr) - set(comb)))
    splits_arr = zip(train_a, test_a)
    print(splits_arr)
    print(len(splits_arr))
    return splits_arr

get_exhaustive_splits(a, test_ratio)
get_exhaustive_splits(b, test_ratio)
get_exhaustive_splits(c, test_ratio)

## Experiment: Subject and Agent features, phases versus no phase

### Test plan

In [ ]:
test_plan_params = {
    'classifier': ['forest', 'jncc2', 'gnb', 'svm', 'forest'],
    'subject': ['doctor+agent', 'doctor', 'agent'],
    'prediction_task': ['presence', 'copresence'],
    'phases': ['nophase-a', '157015-a'],
    'upsampling': [None, 'imb_random', 'imb_smotenc']
}

### Run

In [ ]:
dh_phases = run_test_plan(test_plan_params, '_new_dataset-a', 'phase-nophase_2-classes-uniform', 3, 
                          n_bins=2, bin_strategy='uniform', n_splits=100, test_set=0.1, verbose=1)

In [ ]:
dh = DataHandler('_new_dataset-a', 'phase-nophase_2-classes-uniform', 0)
df = dh.load_obj('results', 'scores')
df = df.dropna(axis='rows', how='all')
 
plot_results(dh, df, 'Phases vs no phase', 0, 3)

In [ ]:
print(pd.__version__)


# Verbal, non-verbal, multi-modal and multi-modal + duration

### Test plan definition

In [ ]:
test_plan_params_modes = {
    'classifier': [ 'gnb'],
    'subject': ['doctor', 'agent', 'doctor+agent'],
    'prediction_task': ['presence', 'copresence'],
    'modes': ['verbal', 'nonverbal', 'multimodal']
}

### Run

In [ ]:
dh_modes = run_test_plan(test_plan_params_modes, '_new_dataset', 'modes_2-classes-uniform_answersDelay', 0, n_bins=2, bin_strategy='uniform', n_splits=100, test_set=0.1)

In [ ]:
dh_modes = DataHandler('_new_dataset', 'modes_2-classes-uniform_answersDelay', 0)
scores_df = dh_modes.load_obj('results', 'scores')
scores_df = scores_df.dropna(axis='rows', how='all')
 
plot_results(dh_modes, scores_df, 'Verbal, non-verbal and multimodal features', 0, 3)

## Experiment: feature selection

In [ ]:
test_plan_params = {
    'classifier': ['gnb', 'fs_rfe-svm-l2_gnb', 'fs_svm-l1_gnb', 'fs_kb5-chi2_gnb', 'fs_kb5-mi_gnb'],
    'subject': ['doctor+agent'],
    'prediction_task': ['presence', 'copresence'],
    'phases': ['157015-a', 'nophase-a']
}
# using NB-G as reference for comparison

In [ ]:
RANDOM_STATE=42
dh = run_test_plan(test_plan_params, 'feature_selection', 'bins_2_uniform_100repeats', 1, n_bins=2, bin_strategy='uniform', test_set=0.1, n_splits=10)

In [ ]:
dh = DataHandler('feature_selection', 'bins_2_uniform_100repeats', 0)
df = dh_modes.load_obj('results', 'scores')
df = df.dropna(axis='rows', how='all')
 
plot_results(dh, df, 'Automatic Feature Selection', 0, 3)

# Learning curves


In [ ]:
from sklearn.model_selection import learning_curve

test_plan_params = {
    'classifier': ['forest'],
    'subject': ['doctor+agent', 'doctor', 'agent'],
    'target': ['presence', 'copresence'],
    'modes': ['verbal', 'nonverbal', 'multimodal']
}
test_plan = list(ParameterGrid(test_plan_params))

forest = RandomForestClassifier(random_state=RANDOM_STATE, oob_score=True)
svm = make_pipeline(StandardScaler(), SVC(random_state=RANDOM_STATE, probability=True))

# whether to apply 'original' upsampling method
upsample = True

cols= 6
fig, ax = plt.subplots(int(len(test_plan)/cols), cols, sharex=True, sharey=True, figsize=(30,15))

results = {}

for idx, test in enumerate(test_plan):
    print("Test %d : %s" % (idx, test))
    

    subject = test['subject']
    target = test['target']
    modes = test['modes']
    classifier = test['classifier']
    clf = forest if classifier == 'forest' else svm

    if subject == 'doctor':
        df = subject_np_df
        if modes == 'verbal':
            feats = subject_features_verbal
        elif modes == 'nonverbal':
            feats = subject_features_nonverbal
        elif modes == 'multimodal':
            feats = subject_features_multimodal
        elif modes == 'multimodal+duration':
            feats = subject_features_multimodal_duration
    elif subject == 'agent':
        df = agent_np_df
        if modes == 'verbal':
            feats = agent_features_verbal
        elif modes == 'nonverbal':
            feats = agent_features_nonverbal
        elif modes == 'multimodal':
            feats = agent_features_multimodal
        elif modes == 'multimodal+duration':
            feats = agent_features_multimodal_duration        
    else:
        df = all_np_df
        if modes == 'verbal':
            feats = all_features_verbal
        elif modes == 'nonverbal':
            feats = all_features_nonverbal
        elif modes == 'multimodal':
            feats = all_features_multimodal
        elif modes == 'multimodal+duration':
            feats = all_features_multimodal_duration        
    
    X, y = prepare_train_data(samples=df, features=feats, target_model=target, upsample=upsample) 
    
    isDoc = 'doctor' in subject
    isAgent = 'agent' in subject
    presidx = 'Presence' if target == 'presence' else 'Co-Presence'
    if subject == 'doctor':
        subjidx = 'Doctor'
    elif subject == 'agent':
        subjidx = 'Agent'
    else:
        subjidx = 'Doctor+Agent'
    if modes == 'verbal':
        phidx = 'Verbal'
    elif modes == 'nonverbal':
        phidx = 'Non-Verbal'
    elif modes == 'multimodal':
        phidx = 'Multimodal'
    elif modes == 'multimodal+duration':
        phidx = 'Multimodal+Duration'
    clfidx = 'Random Forests' if classifier == 'forest' else 'SVM'

    test_set_name = dh.get_grid_name(target=='presence', isDoc, isAgent, modes, classifier)
    
    # create classifier from best params found previously
    best_params = context_df.loc[presidx, subjidx, phidx, clfidx, 'score']['Best Params']
    clf.set_params(**eval(best_params))
    print("  Computing learning curve for best params : %s" % str(best_params))

    train_sizes = np.arange(35, len(df))
    print(train_sizes)
    print(np.unique(y, return_counts=True))
    train_sizes, train_scores, test_scores = learning_curve(clf, 
                                                            X, y, 
                                                            train_sizes=train_sizes, 
                                                            cv=10, 
                                                            scoring='neg_log_loss',
                                                            shuffle=True)

    results[test_set_name] = {}
    results[test_set_name]['train'] = train_scores
    results[test_set_name]['test'] = test_scores
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    cur_ax = ax[int(idx / cols), (idx) % cols]
    cur_ax.plot(train_sizes, train_scores_mean, label='train')
    cur_ax.fill_between(train_sizes, train_scores_mean + train_scores_std,
                    train_scores_mean - train_scores_std, alpha=0.2)

    cur_ax.plot(train_sizes, test_scores_mean, label='valid')
    cur_ax.fill_between(train_sizes, test_scores_mean + test_scores_std,
                    test_scores_mean - test_scores_std, alpha=0.2)
    cur_ax.set_title(test_set_name)
    #cur_ax.xlabel("Count of samples")
    #cur_ax.ylabel("Negative log loss")
    if idx == 0: cur_ax.legend()
fig.text(0.5, 0.1, 'count of data samples', ha='center', va='center')
fig.text(0.1, 0.5, 'negative log loss', ha='center', va='center', rotation='vertical')
dh.save_fig('learning_curves_verbal-nonverbal_forest.png') 
plt.show()

In [ ]:
dh.save_obj(results, 'learning_curves', 'forest')

For non-verbal case we only have 2 features, we have then a chance to vizualize the classification results for some classifiers.

In [ ]:
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA


df = subject_p_df
feats = all_features
target = 'presence'

n_classes = 3
cmap = plt.cm.RdYlBu
plot_step = 0.02  # fine step width for decision surface contours
plot_step_coarser = 0.5  # step widths for coarse classifier guesses

#best_params = context_df.loc['Presence', 'Doctor', 'Non-Verbal', 'Random Forests', 'score']['Best Params']
#forest.set_params(**eval(best_params))
forest = RandomForestClassifier(n_estimators=1)
X, y = prepare_train_data(samples=df, features=feats, target_model=target, upsample='imb_smote', test_set=False)
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)
forest.fit(X_reduced,y)

model = forest
x_min, x_max = X_reduced[:, 0].min() - 1, X_reduced[:, 0].max() + 1
y_min, y_max = X_reduced[:, 1].min() - 1, X_reduced[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                     np.arange(y_min, y_max, plot_step))

# alpha blend the
# decision surfaces of the ensemble of classifiers

# Choose alpha blend level with respect to the number
# of estimators
# that are in use (noting that AdaBoost can use fewer estimators
# than its maximum if it achieves a good enough fit early on)
estimator_alpha = 1.0 / len(model.estimators_)
for idx, tree in enumerate(model.estimators_):
    print idx
    Z = tree.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    cs = plt.contourf(xx, yy, Z, alpha=estimator_alpha, cmap=cmap)

# Build a coarser grid to plot a set of ensemble classifications
# to show how these are different to what we see in the decision
# surfaces. These points are regularly space and do not have a
# black outline
xx_coarser, yy_coarser = np.meshgrid(
    np.arange(x_min, x_max, plot_step_coarser),
    np.arange(y_min, y_max, plot_step_coarser))
Z_points_coarser = model.predict(np.c_[xx_coarser.ravel(),
                                 yy_coarser.ravel()]
                                 ).reshape(xx_coarser.shape)
cs_points = plt.scatter(xx_coarser, yy_coarser, s=15,
                        c=Z_points_coarser, cmap=cmap,
                        edgecolors="none")

# Plot the training points, these are clustered together and have a
# black outline
plt.scatter(X[:, 0], X[:, 1], c=y,
            cmap=ListedColormap(['r', 'y', 'b']),
            edgecolor='k', s=20)
plt.xlabel(feats[0])
plt.ylabel(feats[1])
plt.show()

In [ ]:
## Overfitting

In [ ]:
from sklearn.decomposition import PCA

def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy


def plot_contours(clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, **params)
    
df = all_p_df
feats = get_features_set('Doctor', '157015')
target = 'presence'

n_classes = 2
cmap = plt.cm.RdYlBu
plot_step = 0.02  # fine step width for decision surface contours
plot_step_coarser = 0.5  # step widths for coarse classifier guesses

#best_params = context_df.loc['Presence', 'Doctor', 'Non-Verbal', 'SVM', 'score']['Best Params']
#svm.set_params(**eval(best_params))

splits = prepare_train_data(samples=df, features=feats, prediction_task=target, upsample='imb_random', test_set=0.1, n_splits=1, bins=n_classes, strategy='uniform', verbose=1) 
(X, y, X_test, y_test ) = splits[0]

svm = make_pipeline(StandardScaler(), SVC(random_state=RANDOM_STATE))
param_grid_svm = {
    'svc__kernel': ['linear', 'rbf', 'sigmoid'],
    'svc__C': np.logspace(-2, 1, 4),
    'svc__gamma': np.logspace(-3, 1, 5)
}
grid = gridsearch(svm, 
                  X, y, 
                  'presence', 
                  param_grid=param_grid_svm, 
                  features=feats, 
                  verbose=verbose)
best_params = grid.best_params_
single_best_score = grid.best_score_

print('  Gridsearch evaluated best score %s' % (single_best_score))
print('    Best params %s ...' % (best_params))

In [ ]:

fig = plt.figure(figsize=(20,10))
Cs = [0.0001, 0.001, 0.01, 1, 10, 100]
gammas = [0.001, 0.01, 0.1, 0.1, 1, 10]
for idx, (C, gamma) in enumerate(zip(Cs, gammas)):

    plt.subplot(2,3,idx+1)
    
    svm = SVC(C=C, gamma=gamma, kernel='rbf', random_state=RANDOM_STATE)


    scaler = StandardScaler().fit(X)
    X_ = scaler.transform(X)
    pca = PCA(n_components=2).fit(X_)
    X_reduced = pca.transform(X_)
    svm.fit(X_reduced,y)
    X_test_ = scaler.transform(X_test)
    X_test_reduced = pca.transform(X_test_)
    y_pred = svm.predict(X_test_reduced)
    train_score = balanced_accuracy_score(y, svm.predict(X_reduced))
    test_score = balanced_accuracy_score(y_test, svm.predict(X_test_reduced))
    scores = 'train %.02f test %.02f' % (train_score, test_score)

    #print('TRAIN ' + str(svm.score(X_reduced, y)))

    #fix, ax = plt.subplots(figsize=(5,5))

    # Set-up 2x2 grid for plotting.
    plt.subplots_adjust(wspace=0.4, hspace=0.4)

    X0, X1 = X_reduced[:, 0], X_reduced[:, 1]
    xx, yy = make_meshgrid(X0, X1)

    plot_contours(svm, xx, yy,cmap=plt.cm.coolwarm, alpha=0.6)
    sct = plt.scatter(X0[y==0], X1[y==0], c='r', cmap=plt.cm.coolwarm, s=40, edgecolors='k', label='train class 1')
    sct = plt.scatter(X0[y==1], X1[y==1], c='b', cmap=plt.cm.coolwarm, s=40, edgecolors='k', label='train class 2')

    y_pred = svm.predict(X_test_reduced)
    X2, X3 = X_test_reduced[:, 0], X_test_reduced[:, 1]

    sct2 = plt.scatter(X2[y_test==0], X3[y_test==0], c='r', cmap=plt.cm.coolwarm, s=80, marker='x', edgecolors='k', label='test class 1')
    sct2 = plt.scatter(X2[y_test==1], X3[y_test==1], c='b', cmap=plt.cm.coolwarm, s=80, marker='x', edgecolors='k', label='test class 2')

    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

    plt.xticks(())
    plt.yticks(())
    plt.title('C=%f - %s' %(C, scores))
    if idx==0:
        plt.xlabel('PCA[0]')
        plt.ylabel('PCA[1]')        
        plt.legend()
plt.show()


### phase / no phase

In [ ]:
# Graphs for SVM phases vs no phase, Presence and Co-Presence

dh = DataHandler('1-subject-agent_phases-nophase', 2)
scores_df_index = [['Presence', 'Co-Presence'], 
             ['Doctor', 'Agent', 'Doctor+Agent'], 
             ['No Phase', '157015'], 
             ['Random Forests', 'SVM'],
             ['score', 'err']
            ]
scores_df_columns = ['precision', 'f1', 'recall', 'params']
scores_df_multiindex = pd.MultiIndex.from_product(scores_df_index, 
                                                  names=['Target', 'Subject', 'Phases', 'Classifier', 'Metric'])
scores_df = dh.load_obj('results', 'scores')
if scores_df is None:
    print("No data to load, nothing to draw")

context_df = pd.read_csv(os.path.join(dh.root_path, 'context.csv'))
if context_df is None:
    print("missing contextual info context.pkl")
    
context_df.index = scores_df_multiindex
context_df

In [ ]:
test_plan_params = {
    'classifier': ['forest'],
    'subject': ['doctor+agent', 'doctor', 'agent'],
    'target': ['presence', 'copresence'],
    'phases': [(0,1,0), (0.15,0.70,0.15)]
}
test_plan = list(ParameterGrid(test_plan_params))

forest = RandomForestClassifier(random_state=RANDOM_STATE, oob_score=True)
svm = make_pipeline(StandardScaler(), SVC(random_state=RANDOM_STATE, probability=True))

# whether to apply 'original' upsampling method
upsample = True

cols= 6
fig, ax = plt.subplots(int(len(test_plan)/cols), cols, sharex=True, sharey=True, figsize=(30,10))

results = {}

for idx, test in enumerate(test_plan):
    print("Test %d : %s" % (idx, test))
    

    subject = test['subject']
    target = test['target']
    phases = test['phases']
    classifier = test['classifier']
    clf = forest if classifier == 'forest' else svm
    pgrid = param_grid_rf if classifier == 'forest' else param_grid_svm

    if phases is not None and not phases == (0,1,0):
        print("phases " + str(phases))
        if subject == 'doctor':
            df = subject_p_df
            feats = subject_features
        elif subject == 'agent':
            df = agent_p_df
            feats = agent_features
        elif subject == 'doctor+agent':
            df = all_p_df
            feats=all_features
    else:
        print("No phase")
        if subject == 'doctor':
            df = subject_np_df
            feats = subject_features_nophase
        elif subject == 'agent':
            df = agent_np_df
            feats = agent_features_nophase
        elif subject == 'doctor+agent':
            df = all_np_df
            feats=all_features_nophase        
    
    X, y = prepare_train_data(samples=df, features=feats, target_model=target, upsample=upsample) 
    
    isDoc = 'doctor' in subject
    isAgent = 'agent' in subject
    presidx = 'Presence' if target == 'presence' else 'Co-Presence'
    if subject == 'doctor':
        subjidx = 'Doctor'
    elif subject == 'agent':
        subjidx = 'Agent'
    else:
        subjidx = 'Doctor+Agent'
    phidx = 'No Phase' if phases in [None, (0,1,0)] else '%02d%02d%02d' % ( phases[0]*100, phases[1]*100, phases[2]*100 )
    clfidx = 'Random Forests' if classifier == 'forest' else 'SVM'

    test_set_name = dh.get_grid_name(target=='presence', isDoc, isAgent, phases, classifier)
    
    # create classifier from best params found previously
    best_params = context_df.loc[presidx, subjidx, phidx, clfidx, 'score']['Best Params']
    clf.set_params(**eval(best_params))
    print("  Computing learning curve for best params : %s" % str(best_params))

    train_sizes = np.arange(35, len(df))
    print(train_sizes)
    print(np.unique(y, return_counts=True))
    train_sizes, train_scores, test_scores = learning_curve(clf, 
                                                            X, y, 
                                                            train_sizes=train_sizes, 
                                                            cv=10, 
                                                            scoring='neg_log_loss',
                                                            shuffle=True)

    results[test_set_name] = {}
    results[test_set_name]['train'] = train_scores
    results[test_set_name]['test'] = test_scores
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    cur_ax = ax[int(idx / cols), (idx) % cols]
    cur_ax.plot(train_sizes, train_scores_mean, label='train')
    cur_ax.fill_between(train_sizes, train_scores_mean + train_scores_std,
                    train_scores_mean - train_scores_std, alpha=0.2)

    cur_ax.plot(train_sizes, test_scores_mean, label='valid')
    cur_ax.fill_between(train_sizes, test_scores_mean + test_scores_std,
                    test_scores_mean - test_scores_std, alpha=0.2)
    cur_ax.set_title(test_set_name)
    #cur_ax.xlabel("Count of samples")
    #cur_ax.ylabel("Negative log loss")
    if idx == 0: cur_ax.legend()
fig.text(0.5, 0.09, 'count of data samples', ha='center', va='center')
fig.text(0.1, 0.5, 'negative log loss', ha='center', va='center', rotation='vertical')
dh.save_fig('learning_curves_phase-nophase_forest.png') 
plt.show()

In [ ]:
dh.save_obj(results, 'learning_curves', 'forest')

In [ ]:
plt.show()

In [ ]:
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.plot(train_sizes, train_scores_mean, label='train')
plt.fill_between(train_sizes, train_scores_mean + train_scores_std,
                train_scores_mean - train_scores_std, alpha=0.2)

plt.plot(train_sizes, test_scores_mean, label='valid')
plt.fill_between(train_sizes, test_scores_mean + test_scores_std,
                test_scores_mean - test_scores_std, alpha=0.2)
plt.legend()
plt.show()

In [ ]:
from sklearn.model_selection import validation_curve

train_scores, valid_scores = validation_curve(clf, X, y, "n_estimators",
                                               np.arange(1, 300, 10),
                                               cv=10)

In [ ]:
test_scores = valid_scores
x_range = np.arange(1,300,10)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.plot(x_range, train_scores_mean, label='train')
plt.fill_between(x_range, train_scores_mean + train_scores_std,
                train_scores_mean - train_scores_std, alpha=0.2)

plt.plot(x_range, test_scores_mean, label='valid')
plt.fill_between(x_range, test_scores_mean + test_scores_std,
                test_scores_mean - test_scores_std, alpha=0.2)
plt.legend()
plt.show()

In [ ]:
from mlxtend.plotting import plot_decision_regions

plot_decision_regions(X=X, y=y, clf=clf, legend=2)
plt.xlabel("x", size=5)
plt.ylabel("y", size=5)
plt.title('SVM Decision Region Boundary', size=6)
plt.show()

# Features importance experiments

We would like to check variance and quality of this estimator.
For this we first experiment cross validations with searching also n_estimators, or fixing it with a high value, and check if and how best parameters vary.
Then we will check for a specific set of params, how results and feature importance vary upon repeated runs.
Depending on results, we will compute feature importance for all test plan.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

if 'PresenceClass_5_kmeans' in all_p_df.columns:
    all_p_df = all_p_df.drop(['PresenceClass_5_kmeans'], axis=1)
    
dataset = all_p_df
featureset = get_features_set('Doctor', '157015')
print(featureset)
ds_splits = prepare_train_data(dataset, featureset, 
                                          'presence', bins=5, strategy='kmeans', upsample='imb_smotenc', test_set=0.1, n_splits=1, arff='test1')
print('Generated %d splits' % len(ds_splits))
for (X, y, X_test, y_test) in ds_splits:
    print(np.unique(y, return_counts=True))
    print(np.unique(y_test, return_counts=True))

    kmeans = KMeans(n_clusters=3).fit(X)
    k_test_labels = kmeans.predict(X_test)
    
    pca = PCA(n_components=2)
    X_ = pca.fit_transform(X)
    X_test_ = pca.transform(X_test)
    print(X_.shape)
    plt.scatter(X_[:, 0], X_[:, 1], c=y, cmap='viridis', marker='o')
    plt.scatter(X_test_[:, 0], X_test_[:, 1], c=y_test, cmap='viridis', marker='s')
    plt.show()

## Feature selection by feature clustering and removal of correlated features

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.stats import pearsonr

markers = ['s', 'o', '8', 'p', 'P', '1']
dataset = all_p_df
featureset = get_features_set('Doctor+Agent', 'No Phase-FULL')
print(featureset)
print(featureset.values.shape)
ds_splits = prepare_train_data(dataset, featureset, 
                               'presence', bins=2, strategy='kmeans', upsample='imb_smotenc', test_set=0.1, n_splits=1)

def get_categorical_features(X):
    if 'Expert' in X.columns:
        return ['Expert']
    else:
        return []
    



print('Generated %d splits' % len(ds_splits))
for (X, y, X_test, y_test) in ds_splits:
    print(X.shape)
    # revert to cluster features instead of samples
    ct = make_column_transformer(
        (PassthroughTransformer(), get_categorical_features), # default FunctionTransformer implements identity
        remainder=CustomStandardScaler()
    )
    display(HTML(X.to_html()))
    X_scaled = ct.fit_transform(X)
    print(ct)
    feature_names = parse_feature_names(ct.get_feature_names())
    print(feature_names)
    display(HTML(pd.DataFrame(X_scaled, columns=feature_names).to_html()))
    X = X_scaled.T
    #X['Expert'] = get_features_set('Doctor+Agent', 'No Phase-FULL').loc['Expert'] 
    print(X.shape)
    display(HTML(pd.DataFrame(X, index=feature_names).to_html()))
    print(np.unique(y, return_counts=True))
    print(np.unique(y_test, return_counts=True))

    
    pca = PCA(n_components=2)
    X_ = pca.fit_transform(X)
    #X_test_ = pca.transform(X_test)
    
    kmeans = KMeans(n_clusters=3).fit(X_)
    y_ = kmeans.predict(X_)
    #y_test_ = kmeans.predict(X_test_)
    
    plt.scatter(X_[:, 0], X_[:, 1], c=y_, marker='o')
    for i in np.unique(kmeans.labels_):
        group_feats = list(featureset[y_ == i])
        group_feats_corr_score = pd.DataFrame(np.zeros((len(group_feats))), index=group_feats).T
        print('group #%d : %s' % (i, group_feats))
        for feat1 in group_feats:
            for feat2 in group_feats:
                if feat1 is not feat2:
                    idx_feat1 = list(featureset).index(feat1)
                    idx_feat2 = list(featureset).index(feat2)
                    print('%d %d' % (idx_feat1, idx_feat2))
                    print(X[idx_feat1])
                    print(X[idx_feat2])
                    corr_score = pearsonr(X[idx_feat1], X[idx_feat2])[0]
                    print('Pearson %s vs %s : %f' % (feat1, feat2, corr_score))
                    group_feats_corr_score[feat1] += np.abs(corr_score)
                    group_feats_corr_score[feat2] += np.abs(corr_score)
        display(HTML(group_feats_corr_score.to_html()))
        print('feature to keep: %s' % group_feats[np.argmin(group_feats_corr_score.values)])
        
    print(zip(list(featureset), y_))
    
    """print(X_.shape)
    plt.scatter(X_[:, 0], X_[:, 1], c=y, cmap='viridis', marker='o')
    plt.scatter(X_test_[:, 0], X_test_[:, 1], c=y_test, cmap='viridis', marker='s')
    plt.show()
    
    for idx, label in enumerate(np.unique(k_test_labels)):
        plt.scatter()"""

In [ ]:
def parse_feature_names(feature_names):
    reg = r'.*__(.*)'
    return [re.findall(reg, feat)[0] for feat in feature_names]

print(ct.get_feature_names())
parse_feature_names(ct.get_feature_names())